# Import Library

In [14]:
import pandas as pd
import numpy as np
import re
from pymongo import MongoClient
import sys
import os
import ast
from tabulate import tabulate
import torch

code_path = os.path.abspath('../')
sys.path.append(code_path)

from credentials import connection_string

In [15]:
database_name = "Jakarta_Utara"

# Import Data

## Connect to MongoDB Cluster

In [16]:
collection_name = f"{database_name}_Raw"

In [17]:
client = MongoClient(connection_string)
db = client[database_name]
collection = db[collection_name]

## Get All Documents from Selected Collection

In [18]:
all_documents = collection.find()

## Convert to DataFrame

In [19]:
df = pd.DataFrame(list(all_documents))

# drop _id
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

In [20]:
df

,status_perkara,nomor_perkara,klasifikasi_perkara,tanggal_pendaftaran,lama_proses,terdakwa,penuntut_umum,hakim,jumlah_saksi,putusan_hukuman,barang_bukti,dakwaan
0,Minutasi,734/Pid.B/2024/PN Jkt.Utr,Pencurian,"Senin, 19 Agu. 2024",29 Hari,['ALAMSYAH alias ALAM bin AMIR ULUNG'],"['Rachman Rajasa, S.H.', 'Azhary Arsyad Sulaim...","['SONTAN MERAUKE SINAGA~Hakim Ketua~Ya', 'SLAM...",2,['ALAMSYAH alias ALAM bin AMIR ULUNG~Pidana Pe...,M E N G A D I L I :\n Menyatakan terd...,c\nDakwaan\n:\n Pertama :\n-----------Bahwa...
1,Minutasi,712/Pid.B/2024/PN Jkt.Utr,Pencurian,"Selasa, 13 Agu. 2024",35 Hari,['AAN NABAWIYAH BIN TARDI'],"['MELDA SIAGIAN, S.H.', 'ARI SULTON ABDULLAH, ...","['TOGI PARDEDE~Hakim Ketua~Ya', 'GEDE SUNARJAN...",2,['AAN NABAWIYAH BIN TARDI~Pidana Penjara Waktu...,M E N G A D I L I :\nMenyatakan Ter...,"Dakwaan\n:\n -----------Bahwa ia, Terdakwa ..."
2,Minutasi,707/Pid.B/2024/PN Jkt.Utr,"Penadahan, Penerbitan, dan Pencetakan","Jumat, 09 Agu. 2024",46 Hari,['DAVID ALVIAN bin SYAHRUDIN'],"['TEDDY ANDRI,SH.MH', 'ERMA OCTORA, SH']","['DENY RISWANTO~Hakim Ketua~Ya', 'HANIFZAR~Hak...",2,['DAVID ALVIAN bin SYAHRUDIN~Pidana Penjara Wa...,MENGADILI:\nMenyatakan Terdakwa DAVID ALVIAN b...,Bahwa Terdakwa DAVID ALVIAN bin SYAHRUDIN pada...
3,Minutasi,708/Pid.B/2024/PN Jkt.Utr,"Penadahan, Penerbitan, dan Pencetakan","Jumat, 09 Agu. 2024",46 Hari,['DODIK HERMAWAN BIN ALM. TARJO BIBIT'],"['TEDDY ANDRI,SH.MH', 'DANA MAHENDRA, SH']","['DENY RISWANTO~Hakim Ketua~Ya', 'HANIFZAR~Hak...",1,['DODIK HERMAWAN BIN ALM. TARJO BIBIT~Pidana P...,MENGADILI:\nMenyatakan Terdakwa DODIK HERMAWAN...,D A K W A A N\n------- Bahwa Terdakwa DODIK HE...
4,Minutasi,697/Pid.B/2024/PN Jkt.Utr,Penipuan,"Rabu, 07 Agu. 2024",29 Hari,['ERWIN Bin RADI'],"['DAWIN SOFIAN GAJA, S.H.', 'SADIQA AMALIA, S.H']","['SLAMET WIDODO~Hakim Ketua~Ya', 'IWAN IRAWAN~...",2,['ERWIN Bin RADI~Pidana Penjara Waktu Tertentu...,"Menyatakan Terdakwa Erwin Bin Radi, telah terb...",Pertama :\n-----------Bahwa terdakwa ERWIN Bin...
...,...,...,...,...,...,...,...,...,...,...,...,...
1181,Minutasi,750/Pid.B/2023/PN Jkt.Utr,"Pengeroyokan yang menyebabkan luka ringan, luk...","Rabu, 09 Agu. 2023",83 Hari,"[ABDUL WAHYUNI bin MUKHSIN, RONI PURKON bin RO...","[DONI BOY FAISAL PANJAITAN, S.H., ARI SULTON A...","[ALOYSIUS PRIHARNOTO BAYUAJI~Hakim Ketua~Ya, R...",2,[RONI PURKON bin ROSIHAN EPENDI~Pidana Penjara...,MENGADILI:\nMenyatakan Terdakwa I Abdul Wahyun...,Pertama :\nPerbuatan Terdakwa tersebut sebagai...
1182,Minutasi,752/Pid.Sus/2023/PN Jkt.Utr,Narkotika,"Rabu, 09 Agu. 2023",113 Hari,[ALDI ERLANGGA bin SABIDIN],"[DANA MAHENDRA, SH, ARIF SURYANA , SH]","[LEBANUS SINURAT~Hakim Ketua~Ya, H. SUTAJI~Hak...",2,[ALDI ERLANGGA bin SABIDIN~Subsider Penjara (6...,Menyatakan Terdakwa Aldi Erlangga Bin Sabidin ...,Primair :\nPerbuatan Terdakwa tersebut sebagai...
1183,Minutasi,738/Pid.B/2023/PN Jkt.Utr,Pencurian,"Selasa, 08 Agu. 2023",35 Hari,"[SUPRIADI BIN RATIM, ANWAR RURI BIN SYARIFUDIN...","[RACHMAN RAJASA, S.H., TRI NURANDI SINAGA, SH]","[SLAMET WIDODO~Hakim Ketua~Ya, YULI EFFENDI~Ha...",2,[SUPRIADI BIN RATIM~Pidana Penjara Waktu Terte...,M E N G A D I L I\n Menyatakan Terdakwa 1. S...,"-----------Bahwa mereka, Terdakwa I SUPRIADI b..."
1184,Minutasi,739/Pid.B/2023/PN Jkt.Utr,"Penadahan, Penerbitan, dan Pencetakan","Selasa, 08 Agu. 2023",91 Hari,[MOH FURQON BIN UCOK ARITONANG],"[ANDRIAN AL MAS'UDI, SH.MH., SHUBHAN NOOR HIDA...","[GEDE SUNARJANA~Hakim Ketua~Ya, TOGI PARDEDE~H...",2,[MOH FURQON BIN UCOK ARITONANG~Pidana Penjara ...,MENGADILI :\nMenyatakan terdakwa MOH FURQON Bi...,PERTAMA\nPerbuatan Terdakwa sebagaimana diatur...


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1186 entries, 0 to 1185
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   status_perkara       1186 non-null   object
 1   nomor_perkara        1186 non-null   object
 2   klasifikasi_perkara  1186 non-null   object
 3   tanggal_pendaftaran  1186 non-null   object
 4   lama_proses          1186 non-null   object
 5   terdakwa             1186 non-null   object
 6   penuntut_umum        1186 non-null   object
 7   hakim                1186 non-null   object
 8   jumlah_saksi         1186 non-null   int64 
 9   putusan_hukuman      1186 non-null   object
 10  barang_bukti         1186 non-null   object
 11  dakwaan              1186 non-null   object
dtypes: int64(1), object(11)
memory usage: 111.3+ KB


# Data Understanding

In [22]:
df.head(5)

,status_perkara,nomor_perkara,klasifikasi_perkara,tanggal_pendaftaran,lama_proses,terdakwa,penuntut_umum,hakim,jumlah_saksi,putusan_hukuman,barang_bukti,dakwaan
0,Minutasi,734/Pid.B/2024/PN Jkt.Utr,Pencurian,"Senin, 19 Agu. 2024",29 Hari,['ALAMSYAH alias ALAM bin AMIR ULUNG'],"['Rachman Rajasa, S.H.', 'Azhary Arsyad Sulaim...","['SONTAN MERAUKE SINAGA~Hakim Ketua~Ya', 'SLAM...",2,['ALAMSYAH alias ALAM bin AMIR ULUNG~Pidana Pe...,M E N G A D I L I :\n Menyatakan terd...,c\nDakwaan\n:\n Pertama :\n-----------Bahwa...
1,Minutasi,712/Pid.B/2024/PN Jkt.Utr,Pencurian,"Selasa, 13 Agu. 2024",35 Hari,['AAN NABAWIYAH BIN TARDI'],"['MELDA SIAGIAN, S.H.', 'ARI SULTON ABDULLAH, ...","['TOGI PARDEDE~Hakim Ketua~Ya', 'GEDE SUNARJAN...",2,['AAN NABAWIYAH BIN TARDI~Pidana Penjara Waktu...,M E N G A D I L I :\nMenyatakan Ter...,"Dakwaan\n:\n -----------Bahwa ia, Terdakwa ..."
2,Minutasi,707/Pid.B/2024/PN Jkt.Utr,"Penadahan, Penerbitan, dan Pencetakan","Jumat, 09 Agu. 2024",46 Hari,['DAVID ALVIAN bin SYAHRUDIN'],"['TEDDY ANDRI,SH.MH', 'ERMA OCTORA, SH']","['DENY RISWANTO~Hakim Ketua~Ya', 'HANIFZAR~Hak...",2,['DAVID ALVIAN bin SYAHRUDIN~Pidana Penjara Wa...,MENGADILI:\nMenyatakan Terdakwa DAVID ALVIAN b...,Bahwa Terdakwa DAVID ALVIAN bin SYAHRUDIN pada...
3,Minutasi,708/Pid.B/2024/PN Jkt.Utr,"Penadahan, Penerbitan, dan Pencetakan","Jumat, 09 Agu. 2024",46 Hari,['DODIK HERMAWAN BIN ALM. TARJO BIBIT'],"['TEDDY ANDRI,SH.MH', 'DANA MAHENDRA, SH']","['DENY RISWANTO~Hakim Ketua~Ya', 'HANIFZAR~Hak...",1,['DODIK HERMAWAN BIN ALM. TARJO BIBIT~Pidana P...,MENGADILI:\nMenyatakan Terdakwa DODIK HERMAWAN...,D A K W A A N\n------- Bahwa Terdakwa DODIK HE...
4,Minutasi,697/Pid.B/2024/PN Jkt.Utr,Penipuan,"Rabu, 07 Agu. 2024",29 Hari,['ERWIN Bin RADI'],"['DAWIN SOFIAN GAJA, S.H.', 'SADIQA AMALIA, S.H']","['SLAMET WIDODO~Hakim Ketua~Ya', 'IWAN IRAWAN~...",2,['ERWIN Bin RADI~Pidana Penjara Waktu Tertentu...,"Menyatakan Terdakwa Erwin Bin Radi, telah terb...",Pertama :\n-----------Bahwa terdakwa ERWIN Bin...


**Dataset Features**
| Fitur | Deskripsi | Tipe Data |
|-----------------------|-----------------------------------------------------------------------|:------------:|
| `status_perkara` | Status dari perkara.| `str` |
| `nomor_perkara` | Nomor identifikasi unik untuk perkara.| `str` |
| `klasifikasi_perkara` | Kategori atau jenis perkara.| `str` |
| `tanggal_pendaftaran` | Tanggal ketika perkara didaftarkan di pengadilan.| `str` |
| `lama_proses` | Durasi waktu yang dibutuhkan untuk memproses perkara.| `str` |
| `terdakwa` | Nama individu yang didakwa dalam perkara tersebut.| `list[str]` |
| `penuntut_umum` | Nama jaksa atau penuntut yang menangani perkara.| `list[str]`  |
| `hakim` | Nama hakim yang memimpin sidang perkara.| `list[str]` |
| `jumlah_saksi` | Total jumlah saksi yang dihadirkan dalam sidang.| `int` |
| `putusan_hukuman` | Putusan atau keputusan yang dijatuhkan oleh hakim.| `list[str]` |
| `barang_bukti` | Amar Putusan yang berisi tentang barang bukti perkara.| `str` |
| `dakwaan`| Uraian mengenai tuduhan atau dakwaan yang diajukan terhadap terdakwa.| `str` |







# Data Preprocessing

## Data Cleaning

### Data Cleaning: Null Values

In [23]:
null_counts = df.isnull().sum()

print("Number of null values in each column:")
print(null_counts)

Number of null values in each column:
status_perkara         0
nomor_perkara          0
klasifikasi_perkara    0
tanggal_pendaftaran    0
lama_proses            0
terdakwa               0
penuntut_umum          0
hakim                  0
jumlah_saksi           0
putusan_hukuman        0
barang_bukti           0
dakwaan                0
dtype: int64


In [24]:
null_rows = df[df.isnull().any(axis=1)]

print("Rows with null values:")
print(null_rows)


Rows with null values:
Empty DataFrame
Columns: [status_perkara, nomor_perkara, klasifikasi_perkara, tanggal_pendaftaran, lama_proses, terdakwa, penuntut_umum, hakim, jumlah_saksi, putusan_hukuman, barang_bukti, dakwaan]
Index: []


In [12]:
df_copy = df.dropna()

null_counts = df_copy.isnull().sum()

print("Number of null values in each column:")
print(null_counts)

Number of null values in each column:
status_perkara         0
nomor_perkara          0
klasifikasi_perkara    0
tanggal_pendaftaran    0
lama_proses            0
terdakwa               0
penuntut_umum          0
hakim                  0
jumlah_saksi           0
putusan_hukuman        0
barang_bukti           0
dakwaan                0
dtype: int64


In [25]:
print(f"Jumlah data setelah null dihapus: {df_copy.shape[0]}")

Jumlah data setelah null dihapus: 1186


### Data Cleaning: Data Disamarkan

#### Check Data "Disamarkan"

In [26]:
count_disamarkan = df_copy[df_copy["dakwaan"].str.lower().isin(["disamarkan"])]
print(f"Jumlah data 'Disamarkan': {count_disamarkan.shape[0]}")

Jumlah data 'Disamarkan': 81


In [27]:
print(count_disamarkan)

     status_perkara                nomor_perkara  \
18         Minutasi    656/Pid.B/2024/PN Jkt.Utr   
25         Minutasi    644/Pid.B/2024/PN Jkt.Utr   
34         Minutasi  623/Pid.Sus/2024/PN Jkt.Utr   
38         Minutasi    602/Pid.B/2024/PN Jkt.Utr   
43         Minutasi    608/Pid.B/2024/PN Jkt.Utr   
...             ...                          ...   
1113       Minutasi  834/Pid.Sus/2023/PN Jkt.Utr   
1130       Minutasi    810/Pid.B/2023/PN Jkt.Utr   
1131       Minutasi  811/Pid.Sus/2023/PN Jkt.Utr   
1132       Minutasi  812/Pid.Sus/2023/PN Jkt.Utr   
1143       Minutasi  798/Pid.Sus/2023/PN Jkt.Utr   

                klasifikasi_perkara   tanggal_pendaftaran lama_proses  \
18    Kejahatan terhadap Kesusilaan   Kamis, 25 Jul. 2024     56 Hari   
25    Kejahatan terhadap Kesusilaan  Selasa, 23 Jul. 2024     42 Hari   
34                       Pornografi    Rabu, 17 Jul. 2024     55 Hari   
38    Kejahatan terhadap Kesusilaan   Senin, 15 Jul. 2024     71 Hari   
43    Keja

#### Drop Data "Disamarkan"

In [28]:
df_copy = df_copy[~df_copy["dakwaan"].str.lower().isin(["disamarkan"])]
df_copy.reset_index(drop=True, inplace=True)

In [29]:
count_disamarkan = df_copy[df_copy["dakwaan"].str.lower().isin(["disamarkan"])]

print(f"Jumlah data 'Disamarkan': {count_disamarkan.shape[0]}")

Jumlah data 'Disamarkan': 0


In [30]:
print(f"Jumlah data setelah disamarkan dihapus: {df_copy.shape[0]}")

Jumlah data setelah disamarkan dihapus: 1105


### Data Cleaning: Status Perkara

In [31]:
unique_status_perkara_values = df_copy['status_perkara'].unique()
num_unique_status_perkara = len(unique_status_perkara_values)

print(f"Number of unique values in 'status_perkara': {num_unique_status_perkara}")
print("Unique values:", unique_status_perkara_values)


Number of unique values in 'status_perkara': 1
Unique values: ['Minutasi']


### Data Cleaning: Nomor Perkara

In [32]:
unique_nomor_perkara_values = df_copy['nomor_perkara'].unique()
num_unique_nomor_perkara = len(unique_nomor_perkara_values)

print(f"Jumlah Data: {len(df_copy)}")
print(f"Jumlah Nilai Unique pada Kolom 'nomor_perkara': {num_unique_nomor_perkara}")

Jumlah Data: 1105
Jumlah Nilai Unique pada Kolom 'nomor_perkara': 1105


### Data Cleaning: Klasifikasi Perkara

In [33]:
unique_klasifikasi_perkara_values = df_copy['klasifikasi_perkara'].unique()
num_unique_klasifikasi_perkara = len(unique_klasifikasi_perkara_values)
unique_klasifikasi_counts = df_copy['klasifikasi_perkara'].value_counts()

print(f"Number of unique values in 'status_perkara': {num_unique_klasifikasi_perkara}")
print(unique_klasifikasi_counts)

Number of unique values in 'status_perkara': 30
klasifikasi_perkara
Narkotika                                                                                               419
Pencurian                                                                                               281
Penggelapan                                                                                             101
Penipuan                                                                                                 52
Tindak Pidana Senjata Api atau Benda Tajam                                                               36
Penganiayaan                                                                                             32
Kejahatan Terhadap Keamanan Negara                                                                       26
Kejahatan Perjudian                                                                                      24
Lain-Lain                                                           

### TEMP!!

In [20]:
# df_copy = df_copy[~df_copy["klasifikasi_perkara"].isin(["Lalu Lintas"])]
# df_copy.reset_index(drop=True, inplace=True)

In [21]:
# unique_klasifikasi_perkara_values = df_copy['klasifikasi_perkara'].unique()
# num_unique_klasifikasi_perkara = len(unique_klasifikasi_perkara_values)
# unique_klasifikasi_counts = df_copy['klasifikasi_perkara'].value_counts()

# print(f"Number of unique values in 'status_perkara': {num_unique_klasifikasi_perkara}")
# print(unique_klasifikasi_counts)

### Data Cleaning: Hakim

#### Check "Hakim" Column Data Type (Expected List)

In [34]:
unique_types = df_copy['hakim'].apply(type).unique()
print(f"Unique data types in the column: {unique_types}")

Unique data types in the column: [<class 'str'> <class 'list'>]


In [35]:
count_str_hakim = df_copy[df_copy["hakim"].apply(lambda x: isinstance(x, str))].shape[0]
print(f"Jumlah data dengan 'hakim' berupa string: {count_str_hakim}")

Jumlah data dengan 'hakim' berupa string: 954


#### Convert String to Original Data Type (List)

In [36]:
for index, row in df_copy.iterrows():
    value = row["hakim"]
    
    if isinstance(value, str):
        try:
            print(f"Type: {type(value)}, Row {index} - Value: {value}")
            value = ast.literal_eval(value) 
            df_copy.at[index, "hakim"] = value 
            print(f"Type: {type(value)}, Row {index} - Value: {value}\n")
            
        except (ValueError, SyntaxError):
            print(f"Error in Row {index}: Not a valid list format.")
    

Type: <class 'str'>, Row 0 - Value: ['SONTAN MERAUKE SINAGA~Hakim Ketua~Ya', 'SLAMET WIDODO~Hakim Anggota~Ya', 'IWAN IRAWAN~Hakim Anggota~Ya']
Type: <class 'list'>, Row 0 - Value: ['SONTAN MERAUKE SINAGA~Hakim Ketua~Ya', 'SLAMET WIDODO~Hakim Anggota~Ya', 'IWAN IRAWAN~Hakim Anggota~Ya']

Type: <class 'str'>, Row 1 - Value: ['TOGI PARDEDE~Hakim Ketua~Ya', 'GEDE SUNARJANA~Hakim Anggota~Ya', 'HARTO PANCONO~Hakim Anggota~Ya']
Type: <class 'list'>, Row 1 - Value: ['TOGI PARDEDE~Hakim Ketua~Ya', 'GEDE SUNARJANA~Hakim Anggota~Ya', 'HARTO PANCONO~Hakim Anggota~Ya']

Type: <class 'str'>, Row 2 - Value: ['DENY RISWANTO~Hakim Ketua~Ya', 'HANIFZAR~Hakim Anggota~Ya', 'DIAN ERDIANTO~Hakim Anggota~Ya']
Type: <class 'list'>, Row 2 - Value: ['DENY RISWANTO~Hakim Ketua~Ya', 'HANIFZAR~Hakim Anggota~Ya', 'DIAN ERDIANTO~Hakim Anggota~Ya']

Type: <class 'str'>, Row 3 - Value: ['DENY RISWANTO~Hakim Ketua~Ya', 'HANIFZAR~Hakim Anggota~Ya', 'DIAN ERDIANTO~Hakim Anggota~Ya']
Type: <class 'list'>, Row 3 - Value: [

#### Check Unique Values of Total "Hakim"

In [37]:
unique_types = df_copy['hakim'].apply(len).unique()
print(unique_types)

[ 3  6  9 12]


In [38]:
for index, row in df_copy.iterrows():
    if len(row["hakim"]) == 6: 
        print(f"{index} -> {row['hakim']}")

37 -> ['SYOFIA MARLIANTI TAMBUNAN~Hakim Ketua~Tidak', 'DIAN ERDIANTO~Hakim Anggota~Tidak', 'HOTNAR SIMARMATA~Hakim Anggota~Tidak', 'SYOFIA MARLIANTI TAMBUNAN~Hakim Ketua~Ya', 'DIAN ERDIANTO~Hakim Anggota~Ya', 'HANIFZAR~Hakim Anggota~Ya']
39 -> ['DENY RISWANTO~Hakim Ketua~Tidak', 'HANIFZAR~Hakim Anggota~Tidak', 'MASKUR~Hakim Anggota~Tidak', 'DENY RISWANTO~Hakim Ketua~Ya', 'HANIFZAR~Hakim Anggota~Ya', 'DIAN ERDIANTO~Hakim Anggota~Ya']
40 -> ['R. RUDI KINDARTO~Hakim Ketua~Tidak', 'MARYONO~Hakim Anggota~Tidak', 'ERLY SOELISTYARINI~Hakim Anggota~Tidak', 'R. RUDI KINDARTO~Hakim Ketua~Ya', 'MARYONO~Hakim Anggota~Ya', 'WIJAWIYATA~Hakim Anggota~Ya']
41 -> ['DIAN ERDIANTO~Hakim Ketua~Tidak', 'SYOFIA MARLIANTI TAMBUNAN~Hakim Anggota~Tidak', 'HOTNAR SIMARMATA~Hakim Anggota~Tidak', 'DIAN ERDIANTO~Hakim Ketua~Ya', 'SYOFIA MARLIANTI TAMBUNAN~Hakim Anggota~Ya', 'DENY RISWANTO~Hakim Anggota~Ya']
45 -> ['HANIFZAR~Hakim Ketua~Tidak', 'DENY RISWANTO~Hakim Anggota~Tidak', 'MASKUR~Hakim Anggota~Tidak', 'HAN

In [39]:
for index, row in df_copy.iterrows():
    i = len(row["hakim"])
    for j in range(i):
        print(row["hakim"][j])
    print("")

SONTAN MERAUKE SINAGA~Hakim Ketua~Ya
SLAMET WIDODO~Hakim Anggota~Ya
IWAN IRAWAN~Hakim Anggota~Ya

TOGI PARDEDE~Hakim Ketua~Ya
GEDE SUNARJANA~Hakim Anggota~Ya
HARTO PANCONO~Hakim Anggota~Ya

DENY RISWANTO~Hakim Ketua~Ya
HANIFZAR~Hakim Anggota~Ya
DIAN ERDIANTO~Hakim Anggota~Ya

DENY RISWANTO~Hakim Ketua~Ya
HANIFZAR~Hakim Anggota~Ya
DIAN ERDIANTO~Hakim Anggota~Ya

SLAMET WIDODO~Hakim Ketua~Ya
IWAN IRAWAN~Hakim Anggota~Ya
SONTAN MERAUKE SINAGA~Hakim Anggota~Ya

SONTAN MERAUKE SINAGA~Hakim Ketua~Ya
IWAN IRAWAN~Hakim Anggota~Ya
SLAMET WIDODO~Hakim Anggota~Ya

SLAMET WIDODO~Hakim Ketua~Ya
IWAN IRAWAN~Hakim Anggota~Ya
SONTAN MERAUKE SINAGA~Hakim Anggota~Ya

TOGI PARDEDE~Hakim Ketua~Ya
GEDE SUNARJANA~Hakim Anggota~Ya
HARTO PANCONO~Hakim Anggota~Ya

ERRY IRIAWAN~Hakim Ketua~Ya
EDI JUNAEDI~Hakim Anggota~Ya
YAMTO SUSENA~Hakim Anggota~Ya

GEDE SUNARJANA~Hakim Ketua~Ya
HARTO PANCONO~Hakim Anggota~Ya
TOGI PARDEDE~Hakim Anggota~Ya

HARTO PANCONO~Hakim Ketua~Ya
TOGI PARDEDE~Hakim Anggota~Ya
GEDE SUNARJ

#### Remove Hakim Tidak Aktif

In [40]:
df_copy.loc[:, 'hakim'] = df_copy['hakim'].apply(lambda x: [entry for entry in x if '~Tidak' not in entry])

In [41]:
for index, row in df_copy.iterrows():
    i = len(row["hakim"])
    for j in range(i):
        print(row["hakim"][j])
    print("")

SONTAN MERAUKE SINAGA~Hakim Ketua~Ya
SLAMET WIDODO~Hakim Anggota~Ya
IWAN IRAWAN~Hakim Anggota~Ya

TOGI PARDEDE~Hakim Ketua~Ya
GEDE SUNARJANA~Hakim Anggota~Ya
HARTO PANCONO~Hakim Anggota~Ya

DENY RISWANTO~Hakim Ketua~Ya
HANIFZAR~Hakim Anggota~Ya
DIAN ERDIANTO~Hakim Anggota~Ya

DENY RISWANTO~Hakim Ketua~Ya
HANIFZAR~Hakim Anggota~Ya
DIAN ERDIANTO~Hakim Anggota~Ya

SLAMET WIDODO~Hakim Ketua~Ya
IWAN IRAWAN~Hakim Anggota~Ya
SONTAN MERAUKE SINAGA~Hakim Anggota~Ya

SONTAN MERAUKE SINAGA~Hakim Ketua~Ya
IWAN IRAWAN~Hakim Anggota~Ya
SLAMET WIDODO~Hakim Anggota~Ya

SLAMET WIDODO~Hakim Ketua~Ya
IWAN IRAWAN~Hakim Anggota~Ya
SONTAN MERAUKE SINAGA~Hakim Anggota~Ya

TOGI PARDEDE~Hakim Ketua~Ya
GEDE SUNARJANA~Hakim Anggota~Ya
HARTO PANCONO~Hakim Anggota~Ya

ERRY IRIAWAN~Hakim Ketua~Ya
EDI JUNAEDI~Hakim Anggota~Ya
YAMTO SUSENA~Hakim Anggota~Ya

GEDE SUNARJANA~Hakim Ketua~Ya
HARTO PANCONO~Hakim Anggota~Ya
TOGI PARDEDE~Hakim Anggota~Ya

HARTO PANCONO~Hakim Ketua~Ya
TOGI PARDEDE~Hakim Anggota~Ya
GEDE SUNARJ

#### Check Unique Values after Removing "Hakim Tidak Aktif"

In [42]:
unique_types = df_copy['hakim'].apply(len).unique()
print(unique_types)

[3]


#### Remove Hakim Anggota

In [43]:
df_copy.loc[:, 'hakim'] = df_copy['hakim'].apply(lambda x: [value for value in x if '~Hakim Anggota' not in value])

In [44]:
for index, row in df_copy.iterrows():
    print(row['hakim'])

['SONTAN MERAUKE SINAGA~Hakim Ketua~Ya']
['TOGI PARDEDE~Hakim Ketua~Ya']
['DENY RISWANTO~Hakim Ketua~Ya']
['DENY RISWANTO~Hakim Ketua~Ya']
['SLAMET WIDODO~Hakim Ketua~Ya']
['SONTAN MERAUKE SINAGA~Hakim Ketua~Ya']
['SLAMET WIDODO~Hakim Ketua~Ya']
['TOGI PARDEDE~Hakim Ketua~Ya']
['ERRY IRIAWAN~Hakim Ketua~Ya']
['GEDE SUNARJANA~Hakim Ketua~Ya']
['HARTO PANCONO~Hakim Ketua~Ya']
['HARTO PANCONO~Hakim Ketua~Ya']
['IWAN IRAWAN~Hakim Ketua~Ya']
['I WAYAN GEDE RUMEGA~Hakim Ketua~Ya']
['ERRY IRIAWAN~Hakim Ketua~Ya']
['SLAMET WIDODO~Hakim Ketua~Ya']
['HANIFZAR~Hakim Ketua~Ya']
['DENY RISWANTO~Hakim Ketua~Ya']
['HARTO PANCONO~Hakim Ketua~Ya']
['IWAN IRAWAN~Hakim Ketua~Ya']
['IWAN IRAWAN~Hakim Ketua~Ya']
['YULI SINTHESA TRISTANIA~Hakim Ketua~Ya']
['SYOFIA MARLIANTI TAMBUNAN~Hakim Ketua~Ya']
['R. RUDI KINDARTO~Hakim Ketua~Ya']
['DIAN ERDIANTO~Hakim Ketua~Ya']
['GEDE SUNARJANA~Hakim Ketua~Ya']
['SLAMET WIDODO~Hakim Ketua~Ya']
['DENY RISWANTO~Hakim Ketua~Ya']
['ALOYSIUS PRIHARNOTO BAYUAJI~Hakim Ketua~

#### Check Unique Values after Removing "Hakim Anggota" (Expected 1)

In [45]:
unique_types = df_copy['hakim'].apply(len).unique()
print(unique_types)

[1]


#### Extract Hakim Name

In [46]:
df_copy.loc[:, 'hakim'] = df_copy['hakim'].apply(
    lambda x: [value.split('~Hakim')[0].strip() for value in x]
)

In [47]:
for index, row in df_copy.iterrows():
    print(row['hakim'])

['SONTAN MERAUKE SINAGA']
['TOGI PARDEDE']
['DENY RISWANTO']
['DENY RISWANTO']
['SLAMET WIDODO']
['SONTAN MERAUKE SINAGA']
['SLAMET WIDODO']
['TOGI PARDEDE']
['ERRY IRIAWAN']
['GEDE SUNARJANA']
['HARTO PANCONO']
['HARTO PANCONO']
['IWAN IRAWAN']
['I WAYAN GEDE RUMEGA']
['ERRY IRIAWAN']
['SLAMET WIDODO']
['HANIFZAR']
['DENY RISWANTO']
['HARTO PANCONO']
['IWAN IRAWAN']
['IWAN IRAWAN']
['YULI SINTHESA TRISTANIA']
['SYOFIA MARLIANTI TAMBUNAN']
['R. RUDI KINDARTO']
['DIAN ERDIANTO']
['GEDE SUNARJANA']
['SLAMET WIDODO']
['DENY RISWANTO']
['ALOYSIUS PRIHARNOTO BAYUAJI']
['TOGI PARDEDE']
['TOGI PARDEDE']
['HARTO PANCONO']
['YULI SINTHESA TRISTANIA']
['YAMTO SUSENA']
['TOGI PARDEDE']
['IWAN IRAWAN']
['EDI JUNAEDI']
['SYOFIA MARLIANTI TAMBUNAN']
['SONTAN MERAUKE SINAGA']
['DENY RISWANTO']
['R. RUDI KINDARTO']
['DIAN ERDIANTO']
['TOGI PARDEDE']
['GEDE SUNARJANA']
['SLAMET WIDODO']
['HANIFZAR']
['HARTO PANCONO']
['ERRY IRIAWAN']
['DIAN ERDIANTO']
['HARTO PANCONO']
['MARYONO']
['MARYONO']
['IWAN IR

#### Convert List to String

In [48]:
df_copy['hakim'] = df_copy['hakim'].apply(lambda x: str(x[0]) if isinstance(x, list) and len(x) > 0 else str(x))

In [49]:
unique_types = df_copy['hakim'].apply(type).unique()
print(unique_types)

[<class 'str'>]


In [50]:
for index, row in df_copy.iterrows():
    print(row['hakim'])

SONTAN MERAUKE SINAGA
TOGI PARDEDE
DENY RISWANTO
DENY RISWANTO
SLAMET WIDODO
SONTAN MERAUKE SINAGA
SLAMET WIDODO
TOGI PARDEDE
ERRY IRIAWAN
GEDE SUNARJANA
HARTO PANCONO
HARTO PANCONO
IWAN IRAWAN
I WAYAN GEDE RUMEGA
ERRY IRIAWAN
SLAMET WIDODO
HANIFZAR
DENY RISWANTO
HARTO PANCONO
IWAN IRAWAN
IWAN IRAWAN
YULI SINTHESA TRISTANIA
SYOFIA MARLIANTI TAMBUNAN
R. RUDI KINDARTO
DIAN ERDIANTO
GEDE SUNARJANA
SLAMET WIDODO
DENY RISWANTO
ALOYSIUS PRIHARNOTO BAYUAJI
TOGI PARDEDE
TOGI PARDEDE
HARTO PANCONO
YULI SINTHESA TRISTANIA
YAMTO SUSENA
TOGI PARDEDE
IWAN IRAWAN
EDI JUNAEDI
SYOFIA MARLIANTI TAMBUNAN
SONTAN MERAUKE SINAGA
DENY RISWANTO
R. RUDI KINDARTO
DIAN ERDIANTO
TOGI PARDEDE
GEDE SUNARJANA
SLAMET WIDODO
HANIFZAR
HARTO PANCONO
ERRY IRIAWAN
DIAN ERDIANTO
HARTO PANCONO
MARYONO
MARYONO
IWAN IRAWAN
HARTO PANCONO
DIAN ERDIANTO
EDI JUNAEDI
YAMTO SUSENA
ALOYSIUS PRIHARNOTO BAYUAJI
HANIFZAR
ERRY IRIAWAN
SLAMET WIDODO
MARYONO
I WAYAN GEDE RUMEGA
SLAMET WIDODO
SLAMET WIDODO
YAMTO SUSENA
GEDE SUNARJANA
SONT

In [51]:
print(f"Jumlah data setelah proses cleaning kolom Hakim: {df_copy.shape[0]}")

Jumlah data setelah proses cleaning kolom Hakim: 1105


### Data Cleaning: Barang Bukti

#### RegEx Testing on 1 Data

In [52]:
text = df_copy.iloc[-1]['barang_bukti']
print(text)

Menyatakan Terdakwa I Burhannudin Alias Culik Bin M Rusdi dan Terdakwa II Darwis Abdul Salam Alias Awis Bin Aliman, telah terbukti  secara sah dan meyakinkan bersalah melakukan tindak pidana ”Pencurian dalam keadaan memberatkan” ;
Menjatuhkan pidana terhadap Para Terdakwa oleh karena itu dengan pidana penjara masing-masing selama   1 (satu) tahun dan 2 (dua) bulan;
Menetapkan masa penangkapan dan penahanan yang telah dijalani oleh Para Terdakwa dikurangkan seluruhnya dari pidana yang dijatuhkan;
Menetapkan Para Terdakwa tetap ditahan;
Menetapkan barang bukti  berupa:
1 (satu) unit sepeda motor Yamaha NMAX, nopol : B-3565-UVO, warna biru, Noka : MH3SG5620LJ0693, Nosin : G3E8E0073781, An. TIGOR ESRON FERNANDES;
Dikembalikan kepada koraban Tigor Esron Fernandes
1 (satu) unit sepeda motor Honda Beat, nopol : E-4158-HY.
Dirampas untuk negara;
6. Membebankan kepada Para Terdakwa membayar biaya perkara masing-masing sejumlah Rp.5.000,00- (lima ribu rupiah);


##### Remove Unwanted Symbol and Char

In [53]:
cleaned_text = re.sub(r'[^a-zA-Z0-9., /\\()\"\'\n-]+', '', text)
cleaned_text = re.sub(r'-{2,}', '-', cleaned_text)

In [54]:
print(cleaned_text)

Menyatakan Terdakwa I Burhannudin Alias Culik Bin M Rusdi dan Terdakwa II Darwis Abdul Salam Alias Awis Bin Aliman, telah terbukti  secara sah dan meyakinkan bersalah melakukan tindak pidana Pencurian dalam keadaan memberatkan 
Menjatuhkan pidana terhadap Para Terdakwa oleh karena itu dengan pidana penjara masing-masing selama   1 (satu) tahun dan 2 (dua) bulan
Menetapkan masa penangkapan dan penahanan yang telah dijalani oleh Para Terdakwa dikurangkan seluruhnya dari pidana yang dijatuhkan
Menetapkan Para Terdakwa tetap ditahan
Menetapkan barang bukti  berupa
1 (satu) unit sepeda motor Yamaha NMAX, nopol  B-3565-UVO, warna biru, Noka  MH3SG5620LJ0693, Nosin  G3E8E0073781, An. TIGOR ESRON FERNANDES
Dikembalikan kepada koraban Tigor Esron Fernandes
1 (satu) unit sepeda motor Honda Beat, nopol  E-4158-HY.
Dirampas untuk negara
6. Membebankan kepada Para Terdakwa membayar biaya perkara masing-masing sejumlah Rp.5.000,00- (lima ribu rupiah)


##### Remove Multiple Whitespaces

In [55]:
cleaned_text = re.sub(r' +', ' ', cleaned_text)

In [56]:
print(cleaned_text)

Menyatakan Terdakwa I Burhannudin Alias Culik Bin M Rusdi dan Terdakwa II Darwis Abdul Salam Alias Awis Bin Aliman, telah terbukti secara sah dan meyakinkan bersalah melakukan tindak pidana Pencurian dalam keadaan memberatkan 
Menjatuhkan pidana terhadap Para Terdakwa oleh karena itu dengan pidana penjara masing-masing selama 1 (satu) tahun dan 2 (dua) bulan
Menetapkan masa penangkapan dan penahanan yang telah dijalani oleh Para Terdakwa dikurangkan seluruhnya dari pidana yang dijatuhkan
Menetapkan Para Terdakwa tetap ditahan
Menetapkan barang bukti berupa
1 (satu) unit sepeda motor Yamaha NMAX, nopol B-3565-UVO, warna biru, Noka MH3SG5620LJ0693, Nosin G3E8E0073781, An. TIGOR ESRON FERNANDES
Dikembalikan kepada koraban Tigor Esron Fernandes
1 (satu) unit sepeda motor Honda Beat, nopol E-4158-HY.
Dirampas untuk negara
6. Membebankan kepada Para Terdakwa membayar biaya perkara masing-masing sejumlah Rp.5.000,00- (lima ribu rupiah)


##### Extract Only Barang Bukti

In [57]:
# pattern = r"Menetapkan barang bukti berupa\s*(.*?)(?=\n\s*[A-Z]|$)"
# pattern = r"Menetapkan barang bukti berupa\s*(.*?)(?=\n[A-Z])"
pattern = r"barang bukti berupa\s*(.*)"

match = re.search(pattern, cleaned_text, re.DOTALL | re.IGNORECASE)

if match:
    print(match.group().strip())
    
else:
    print("Tidak ada kecocokan ditemukan.")

barang bukti berupa
1 (satu) unit sepeda motor Yamaha NMAX, nopol B-3565-UVO, warna biru, Noka MH3SG5620LJ0693, Nosin G3E8E0073781, An. TIGOR ESRON FERNANDES
Dikembalikan kepada koraban Tigor Esron Fernandes
1 (satu) unit sepeda motor Honda Beat, nopol E-4158-HY.
Dirampas untuk negara
6. Membebankan kepada Para Terdakwa membayar biaya perkara masing-masing sejumlah Rp.5.000,00- (lima ribu rupiah)


#### RegEx on Entire DataFrame

##### Extract Barang Bukti

In [58]:
# patterns = [
#     r"barang\s*bukti\s*berupa\s*:?\s*(.*?)(?=\n\s*[A-Z]|$)",
#     r"barang\s*bukti,\s*berupa\s*:?\s*(.*?)(?=\n\s*[A-Z]|$)",
#     r"barang\s*bukti\s*:?\s*(.*?)(?=\n\s*[A-Z]|$)",
#     r"barang-bukti\s*:?\s*(.*?)(?=\n\s*[A-Z]|$)"
# ]

patterns = [
    r"barang\s*bukti\s*berupa\s*:?\s*(.*)",
    r"barang\s*bukti,\s*berupa\s*:?\s*(.*)",
    r"barang\s*bukti\s*:?\s*(.*)",
    r"barang-bukti\s*:?\s*(.*)"
]

rows_to_drop = []

for index, row in df_copy.iterrows():
    text_barang_bukti = row['barang_bukti']
    
    cleaned_barang_bukti = re.sub(r'[^a-zA-Z0-9., /\\()\"\'\n-]+', '', text_barang_bukti)
    cleaned_barang_bukti = re.sub(r'-{2,}', '-', cleaned_barang_bukti)
    cleaned_barang_bukti = re.sub(r' +', ' ', cleaned_barang_bukti)
    
    match_found = False

    for pattern in patterns:
        match = re.search(pattern, cleaned_barang_bukti, re.DOTALL | re.IGNORECASE)

        if match:
            df_copy.loc[index, 'cleaned_barang_bukti'] = match.group(1).strip().replace('\n', ' ')
            match_found = True
            break

    
    if not match_found:
        print(f"{index}. Barang Bukti Tidak Ditemukan pada {row['nomor_perkara']}")
        rows_to_drop.append(index) 
        # print(cleaned_barang_bukti)



38. Barang Bukti Tidak Ditemukan pada 607/Pid.Sus/2024/PN Jkt.Utr
99. Barang Bukti Tidak Ditemukan pada 525/Pid.B/2024/PN Jkt.Utr
112. Barang Bukti Tidak Ditemukan pada 521/Pid.B/2024/PN Jkt.Utr
113. Barang Bukti Tidak Ditemukan pada 522/Pid.B/2024/PN Jkt.Utr
224. Barang Bukti Tidak Ditemukan pada 382/Pid.B/2024/PN Jkt.Utr
476. Barang Bukti Tidak Ditemukan pada 92/Pid.B/2024/PN Jkt.Utr
521. Barang Bukti Tidak Ditemukan pada 43/Pid.B/2024/PN Jkt.Utr
550. Barang Bukti Tidak Ditemukan pada 2/Pid.B/2024/PN Jkt.Utr
574. Barang Bukti Tidak Ditemukan pada 1364/Pid.B/2023/PN Jkt.Utr
579. Barang Bukti Tidak Ditemukan pada 1350/Pid.Sus/2023/PN Jkt.Utr
718. Barang Bukti Tidak Ditemukan pada 1187/Pid.Sus/2023/PN Jkt.Utr
769. Barang Bukti Tidak Ditemukan pada 1116/Pid.B/2023/PN Jkt.Utr
773. Barang Bukti Tidak Ditemukan pada 1098/Pid.B/2023/PN Jkt.Utr
870. Barang Bukti Tidak Ditemukan pada 1003/Pid.B/2023/PN Jkt.Utr
910. Barang Bukti Tidak Ditemukan pada 953/Pid.B/2023/PN Jkt.Utr
1045. Barang Bukti 

##### Get Case with No Barang Bukti

In [59]:
true_values = df_copy.isna()
df_fail_barang_bukti = df_copy[true_values['cleaned_barang_bukti']]

In [60]:
df_fail_barang_bukti.head()

,status_perkara,nomor_perkara,klasifikasi_perkara,tanggal_pendaftaran,lama_proses,terdakwa,penuntut_umum,hakim,jumlah_saksi,putusan_hukuman,barang_bukti,dakwaan,cleaned_barang_bukti
38,Minutasi,607/Pid.Sus/2024/PN Jkt.Utr,Narkotika,"Senin, 15 Jul. 2024",50 Hari,['RHAMSES RHAMA DHANI bin HERAKLY MULIA'],"['SHUBHAN NOOR HIDAYAT, SH', 'ARI SULTON ABDUL...",SONTAN MERAUKE SINAGA,2,['RHAMSES RHAMA DHANI bin HERAKLY MULIA~Terdak...,M E N E T A P K A N :\nMenyatakan hak menuntut...,D A K W A A N\n Pertama\n ------- Bahwa Terd...,NaN
99,Minutasi,525/Pid.B/2024/PN Jkt.Utr,Pembunuhan,"Jumat, 21 Jun. 2024",62 Hari,['MOCHAMAD KEVIN AL FAZRI bin MAT DACUNG WIJANA'],"['MELDA SIAGIAN, S.H.', 'SHUBHAN NOOR HIDAYAT,...",TOGI PARDEDE,2,['MOCHAMAD KEVIN AL FAZRI bin MAT DACUNG WIJAN...,MENGADILI :\nMenyatakan Terdakwa. MOCHAMAD KEV...,"DAKWAAN \nPertama :\n-----------Bahwa ia, Terd...",NaN
112,Minutasi,521/Pid.B/2024/PN Jkt.Utr,"Pengeroyokan yang menyebabkan luka ringan, luk...","Kamis, 20 Jun. 2024",75 Hari,['GIDION CARLOS'],"['DONI BOY FAISAL PANJAITAN, SH', 'ARI SULTON ...",R. RUDI KINDARTO,2,['GIDION CARLOS~Pidana Penjara Waktu Tertentu ...,"Menyatakan Terdakwa Gidion Carlos, telah terbu...",Dakwaan : \nPertama :\n------...,NaN
113,Minutasi,522/Pid.B/2024/PN Jkt.Utr,"Pengeroyokan yang menyebabkan luka ringan, luk...","Kamis, 20 Jun. 2024",75 Hari,"[""JOHN'S ANTONIO""]","['DONI BOY FAISAL PANJAITAN, SH', 'ARI SULTON ...",MARYONO,2,"[""JOHN'S ANTONIO~Pidana Penjara Waktu Tertentu...","I\nMenyatakan Terdakwa John's Antonio, telah t...","Dakwaan:\nPertama :\n-----------Bahwa ia, Terd...",NaN
224,Minutasi,382/Pid.B/2024/PN Jkt.Utr,Penganiayaan,"Senin, 13 Mei 2024",45 Hari,"['IRPAN DADI, SE']","['NOFIMAR, S.H.', 'DANA MAHENDRA, SH']",IWAN IRAWAN,2,"['IRPAN DADI, SE~Pidana Penjara Waktu Tertentu...","MENGADILI :\nMenyatakan Terdakwa Irpan Dadi, S...","Dakwaan\nBahwa ia, Terdakwa IRPAN DADI, SE pad...",NaN


In [61]:
df_fail_barang_bukti.to_csv(f'../Data/Misc/{database_name}_kasus tanpa barang bukti.csv')

##### Drop Perkara Tanpa Barang Bukti

In [62]:
df_copy.drop(rows_to_drop, inplace=True)
print(f"rows dropped: {rows_to_drop}")
print(f"rows dropped: {len(rows_to_drop)}")

rows dropped: [38, 99, 112, 113, 224, 476, 521, 550, 574, 579, 718, 769, 773, 870, 910, 1045, 1069, 1083, 1103]
rows dropped: 19


In [63]:
df_copy.head()

,status_perkara,nomor_perkara,klasifikasi_perkara,tanggal_pendaftaran,lama_proses,terdakwa,penuntut_umum,hakim,jumlah_saksi,putusan_hukuman,barang_bukti,dakwaan,cleaned_barang_bukti
0,Minutasi,734/Pid.B/2024/PN Jkt.Utr,Pencurian,"Senin, 19 Agu. 2024",29 Hari,['ALAMSYAH alias ALAM bin AMIR ULUNG'],"['Rachman Rajasa, S.H.', 'Azhary Arsyad Sulaim...",SONTAN MERAUKE SINAGA,2,['ALAMSYAH alias ALAM bin AMIR ULUNG~Pidana Pe...,M E N G A D I L I :\n Menyatakan terd...,c\nDakwaan\n:\n Pertama :\n-----------Bahwa...,berupa 3 (tiga) lembar foto copy STNK mobil bo...
1,Minutasi,712/Pid.B/2024/PN Jkt.Utr,Pencurian,"Selasa, 13 Agu. 2024",35 Hari,['AAN NABAWIYAH BIN TARDI'],"['MELDA SIAGIAN, S.H.', 'ARI SULTON ABDULLAH, ...",TOGI PARDEDE,2,['AAN NABAWIYAH BIN TARDI~Pidana Penjara Waktu...,M E N G A D I L I :\nMenyatakan Ter...,"Dakwaan\n:\n -----------Bahwa ia, Terdakwa ...",1 (satu) unit handpone merek VIVO Type Y27s wa...
2,Minutasi,707/Pid.B/2024/PN Jkt.Utr,"Penadahan, Penerbitan, dan Pencetakan","Jumat, 09 Agu. 2024",46 Hari,['DAVID ALVIAN bin SYAHRUDIN'],"['TEDDY ANDRI,SH.MH', 'ERMA OCTORA, SH']",DENY RISWANTO,2,['DAVID ALVIAN bin SYAHRUDIN~Pidana Penjara Wa...,MENGADILI:\nMenyatakan Terdakwa DAVID ALVIAN b...,Bahwa Terdakwa DAVID ALVIAN bin SYAHRUDIN pada...,1 (satu) lembar STNK asli motor Honda Beat Nop...
3,Minutasi,708/Pid.B/2024/PN Jkt.Utr,"Penadahan, Penerbitan, dan Pencetakan","Jumat, 09 Agu. 2024",46 Hari,['DODIK HERMAWAN BIN ALM. TARJO BIBIT'],"['TEDDY ANDRI,SH.MH', 'DANA MAHENDRA, SH']",DENY RISWANTO,1,['DODIK HERMAWAN BIN ALM. TARJO BIBIT~Pidana P...,MENGADILI:\nMenyatakan Terdakwa DODIK HERMAWAN...,D A K W A A N\n------- Bahwa Terdakwa DODIK HE...,1 (satu) lembar STNK asli motor Honda Beat Nop...
4,Minutasi,697/Pid.B/2024/PN Jkt.Utr,Penipuan,"Rabu, 07 Agu. 2024",29 Hari,['ERWIN Bin RADI'],"['DAWIN SOFIAN GAJA, S.H.', 'SADIQA AMALIA, S.H']",SLAMET WIDODO,2,['ERWIN Bin RADI~Pidana Penjara Waktu Tertentu...,"Menyatakan Terdakwa Erwin Bin Radi, telah terb...",Pertama :\n-----------Bahwa terdakwa ERWIN Bin...,1 (satu) buah kunci kontak asli sepeda motor H...


##### Check NaN Values

In [64]:
null_counts = df_copy.isna().sum()

print("Number of null values in each column:")
print(null_counts)

Number of null values in each column:
status_perkara          0
nomor_perkara           0
klasifikasi_perkara     0
tanggal_pendaftaran     0
lama_proses             0
terdakwa                0
penuntut_umum           0
hakim                   0
jumlah_saksi            0
putusan_hukuman         0
barang_bukti            0
dakwaan                 0
cleaned_barang_bukti    0
dtype: int64


In [65]:
print(f"Jumlah data setelah proses cleaning kolom Barang Bukti: {df_copy.shape[0]}")

Jumlah data setelah proses cleaning kolom Barang Bukti: 1086


### Data Cleaning: Dakwaan

#### Test Cleaning on 1 Data

In [66]:
dakwaan = df_copy.iloc[30]['dakwaan']
print(dakwaan)

C
Dakwaan
:
    PERTAMA :
-----------Bahwa ia, Terdakwa DANANG HARIADI bin HARSO bersama-sama dengan saksi DRIA PITAKA alias TOKEM bin MUNAWAR (dilakukan Penuntutan secara terpisah) pada hari Minggu tanggal 10 Maret 2024 sekitar jam 06.30 Wib atau setidak-tidaknya pada waktu lain dalam bulan Maret 2024 atau setidak-tidaknya pada waktu lain dalam tahun 2024 bertempat di Kampung Muara Bahari Gang Bak Air 3 RT.003/013 Kelurahan Tanjung Priok Kecamatan Tanjung Priok Jakarta Utara atau setidak-tidaknya di suatu tempat yang masih termasuk dalam daerah hukum Pengadilan Negeri Jakarta Utara  “melakukan permufakatan jahat dan tanpa hak atau melawan hukum menawarkan untuk dijual, menjual, membeli, menerima, menjadi perantara dalam jual beli, menukar, atau menyerahkan Narkotika Golongan I, yang dilakukan Terdakwa dengan cara sebagai berikut:-------------------------------------------------------------------------
--Bahwa pada hari Sabtu tanggal 09 Maret 2024 sekira jam 17.30 WIB, Sdr. BANG KUPLUK

##### Clean Unwanted Char and Symbols

In [67]:
# cleaning_dakwaan_unwanted_char = re.findall(r'[a-zA-Z0-9., /\\()\"\'\n]+', dakwaan)
# cleaned_dakwaan = ''.join(cleaning_dakwaan_unwanted_char)
# print(cleaned_dakwaan)

cleaning_dakwaan_unwanted_char = re.findall(r'[a-zA-Z0-9., /\\()\"\'\n-]+', dakwaan)

cleaned_dakwaan = ''.join(cleaning_dakwaan_unwanted_char)
cleaned_dakwaan = re.sub(r'-{2,}', '-', cleaned_dakwaan)
cleaned_dakwaan = re.sub(r' +', ' ', cleaned_dakwaan)

print(cleaned_dakwaan)

C
Dakwaan

 PERTAMA 
-Bahwa ia, Terdakwa DANANG HARIADI bin HARSO bersama-sama dengan saksi DRIA PITAKA alias TOKEM bin MUNAWAR (dilakukan Penuntutan secara terpisah) pada hari Minggu tanggal 10 Maret 2024 sekitar jam 06.30 Wib atau setidak-tidaknya pada waktu lain dalam bulan Maret 2024 atau setidak-tidaknya pada waktu lain dalam tahun 2024 bertempat di Kampung Muara Bahari Gang Bak Air 3 RT.003/013 Kelurahan Tanjung Priok Kecamatan Tanjung Priok Jakarta Utara atau setidak-tidaknya di suatu tempat yang masih termasuk dalam daerah hukum Pengadilan Negeri Jakarta Utara melakukan permufakatan jahat dan tanpa hak atau melawan hukum menawarkan untuk dijual, menjual, membeli, menerima, menjadi perantara dalam jual beli, menukar, atau menyerahkan Narkotika Golongan I, yang dilakukan Terdakwa dengan cara sebagai berikut-
-Bahwa pada hari Sabtu tanggal 09 Maret 2024 sekira jam 17.30 WIB, Sdr. BANG KUPLUK (belum tertangkap/DPO) datang menemui terdakwa di warung di Kampung Muara Bahari Tanjung P

##### Remove Header

In [68]:
pattern = r"(?i)dakwaan(?!\nnomor)[\s\S]*?(?=(?:dakwaan|$))"

text_dakwaan = cleaned_dakwaan
matches = re.findall(pattern, text_dakwaan, re.DOTALL | re.IGNORECASE)

if matches: 
    last_match = matches[-1]
    cleaned_dakwaan = last_match.strip()
else:
    cleaned_dakwaan = text_dakwaan
    print(f"raw dakwaan copied at {index}")

print(cleaned_dakwaan)

Dakwaan

 PERTAMA 
-Bahwa ia, Terdakwa DANANG HARIADI bin HARSO bersama-sama dengan saksi DRIA PITAKA alias TOKEM bin MUNAWAR (dilakukan Penuntutan secara terpisah) pada hari Minggu tanggal 10 Maret 2024 sekitar jam 06.30 Wib atau setidak-tidaknya pada waktu lain dalam bulan Maret 2024 atau setidak-tidaknya pada waktu lain dalam tahun 2024 bertempat di Kampung Muara Bahari Gang Bak Air 3 RT.003/013 Kelurahan Tanjung Priok Kecamatan Tanjung Priok Jakarta Utara atau setidak-tidaknya di suatu tempat yang masih termasuk dalam daerah hukum Pengadilan Negeri Jakarta Utara melakukan permufakatan jahat dan tanpa hak atau melawan hukum menawarkan untuk dijual, menjual, membeli, menerima, menjadi perantara dalam jual beli, menukar, atau menyerahkan Narkotika Golongan I, yang dilakukan Terdakwa dengan cara sebagai berikut-
-Bahwa pada hari Sabtu tanggal 09 Maret 2024 sekira jam 17.30 WIB, Sdr. BANG KUPLUK (belum tertangkap/DPO) datang menemui terdakwa di warung di Kampung Muara Bahari Tanjung Pri

##### Remove Footer

In [69]:
pattern = r"^(.*?)\s+Jakarta Utara,\s+\d{1,2}\s+\w+\s+\d{4}"

text_dakwaan = cleaned_dakwaan
matches = re.findall(pattern, text_dakwaan, re.DOTALL | re.IGNORECASE)

if matches: 
    last_match = matches[-1]
    cleaned_dakwaan = last_match.strip()
else:
    cleaned_dakwaan = text_dakwaan
    print(f"raw dakwaan copied at {index}")

print(cleaned_dakwaan)

raw dakwaan copied at 1104
Dakwaan

 PERTAMA 
-Bahwa ia, Terdakwa DANANG HARIADI bin HARSO bersama-sama dengan saksi DRIA PITAKA alias TOKEM bin MUNAWAR (dilakukan Penuntutan secara terpisah) pada hari Minggu tanggal 10 Maret 2024 sekitar jam 06.30 Wib atau setidak-tidaknya pada waktu lain dalam bulan Maret 2024 atau setidak-tidaknya pada waktu lain dalam tahun 2024 bertempat di Kampung Muara Bahari Gang Bak Air 3 RT.003/013 Kelurahan Tanjung Priok Kecamatan Tanjung Priok Jakarta Utara atau setidak-tidaknya di suatu tempat yang masih termasuk dalam daerah hukum Pengadilan Negeri Jakarta Utara melakukan permufakatan jahat dan tanpa hak atau melawan hukum menawarkan untuk dijual, menjual, membeli, menerima, menjadi perantara dalam jual beli, menukar, atau menyerahkan Narkotika Golongan I, yang dilakukan Terdakwa dengan cara sebagai berikut-
-Bahwa pada hari Sabtu tanggal 09 Maret 2024 sekira jam 17.30 WIB, Sdr. BANG KUPLUK (belum tertangkap/DPO) datang menemui terdakwa di warung di Kampu

#### Cleaning on Entire DataFrame

##### Clean Unwanted Char & Symbols

In [70]:
def clean_dakwaan(text):
    cleaning_dakwaan_unwanted_char = re.findall(r'[a-zA-Z0-9., /\\()\"\'\n-]+', text)
    cleaned_dakwaan = ''.join(cleaning_dakwaan_unwanted_char)
    
    cleaned_dakwaan = re.sub(r'-{2,}', '-', cleaned_dakwaan)
    cleaned_dakwaan = re.sub(r' +', ' ', cleaned_dakwaan)
    
    return cleaned_dakwaan

In [71]:
df_copy['cleaned_dakwaan'] = df_copy['dakwaan'].apply(clean_dakwaan)

In [72]:
df_copy.head()

,status_perkara,nomor_perkara,klasifikasi_perkara,tanggal_pendaftaran,lama_proses,terdakwa,penuntut_umum,hakim,jumlah_saksi,putusan_hukuman,barang_bukti,dakwaan,cleaned_barang_bukti,cleaned_dakwaan
0,Minutasi,734/Pid.B/2024/PN Jkt.Utr,Pencurian,"Senin, 19 Agu. 2024",29 Hari,['ALAMSYAH alias ALAM bin AMIR ULUNG'],"['Rachman Rajasa, S.H.', 'Azhary Arsyad Sulaim...",SONTAN MERAUKE SINAGA,2,['ALAMSYAH alias ALAM bin AMIR ULUNG~Pidana Pe...,M E N G A D I L I :\n Menyatakan terd...,c\nDakwaan\n:\n Pertama :\n-----------Bahwa...,berupa 3 (tiga) lembar foto copy STNK mobil bo...,"c\nDakwaan\n\n Pertama \n-Bahwa ia, Terdakwa A..."
1,Minutasi,712/Pid.B/2024/PN Jkt.Utr,Pencurian,"Selasa, 13 Agu. 2024",35 Hari,['AAN NABAWIYAH BIN TARDI'],"['MELDA SIAGIAN, S.H.', 'ARI SULTON ABDULLAH, ...",TOGI PARDEDE,2,['AAN NABAWIYAH BIN TARDI~Pidana Penjara Waktu...,M E N G A D I L I :\nMenyatakan Ter...,"Dakwaan\n:\n -----------Bahwa ia, Terdakwa ...",1 (satu) unit handpone merek VIVO Type Y27s wa...,"Dakwaan\n\n -Bahwa ia, Terdakwa AAN NABAWIYAH ..."
2,Minutasi,707/Pid.B/2024/PN Jkt.Utr,"Penadahan, Penerbitan, dan Pencetakan","Jumat, 09 Agu. 2024",46 Hari,['DAVID ALVIAN bin SYAHRUDIN'],"['TEDDY ANDRI,SH.MH', 'ERMA OCTORA, SH']",DENY RISWANTO,2,['DAVID ALVIAN bin SYAHRUDIN~Pidana Penjara Wa...,MENGADILI:\nMenyatakan Terdakwa DAVID ALVIAN b...,Bahwa Terdakwa DAVID ALVIAN bin SYAHRUDIN pada...,1 (satu) lembar STNK asli motor Honda Beat Nop...,Bahwa Terdakwa DAVID ALVIAN bin SYAHRUDIN pada...
3,Minutasi,708/Pid.B/2024/PN Jkt.Utr,"Penadahan, Penerbitan, dan Pencetakan","Jumat, 09 Agu. 2024",46 Hari,['DODIK HERMAWAN BIN ALM. TARJO BIBIT'],"['TEDDY ANDRI,SH.MH', 'DANA MAHENDRA, SH']",DENY RISWANTO,1,['DODIK HERMAWAN BIN ALM. TARJO BIBIT~Pidana P...,MENGADILI:\nMenyatakan Terdakwa DODIK HERMAWAN...,D A K W A A N\n------- Bahwa Terdakwa DODIK HE...,1 (satu) lembar STNK asli motor Honda Beat Nop...,D A K W A A N\n- Bahwa Terdakwa DODIK HERMAWAN...
4,Minutasi,697/Pid.B/2024/PN Jkt.Utr,Penipuan,"Rabu, 07 Agu. 2024",29 Hari,['ERWIN Bin RADI'],"['DAWIN SOFIAN GAJA, S.H.', 'SADIQA AMALIA, S.H']",SLAMET WIDODO,2,['ERWIN Bin RADI~Pidana Penjara Waktu Tertentu...,"Menyatakan Terdakwa Erwin Bin Radi, telah terb...",Pertama :\n-----------Bahwa terdakwa ERWIN Bin...,1 (satu) buah kunci kontak asli sepeda motor H...,"Pertama \n-Bahwa terdakwa ERWIN Bin RADI, pada..."


##### Remove Header

In [73]:
pattern = r"(?i)dakwaan(?!\nnomor)[\s\S]*?(?=(?:dakwaan|$))"

for index, row in df_copy.iterrows():
    text_dakwaan = row['cleaned_dakwaan']
    matches = re.findall(pattern, text_dakwaan, re.DOTALL | re.IGNORECASE)

    if matches: 
        last_match = matches[-1]
        df_copy.loc[index, "cleaned_dakwaan"] = last_match.strip()
    else:
        df_copy.loc[index, "cleaned_dakwaan"] = text_dakwaan
        print(f"raw dakwaan copied at {index}")


raw dakwaan copied at 2
raw dakwaan copied at 3
raw dakwaan copied at 4
raw dakwaan copied at 5
raw dakwaan copied at 6
raw dakwaan copied at 7
raw dakwaan copied at 8
raw dakwaan copied at 11
raw dakwaan copied at 12
raw dakwaan copied at 15
raw dakwaan copied at 17
raw dakwaan copied at 18
raw dakwaan copied at 20
raw dakwaan copied at 21
raw dakwaan copied at 24
raw dakwaan copied at 28
raw dakwaan copied at 35
raw dakwaan copied at 37
raw dakwaan copied at 39
raw dakwaan copied at 41
raw dakwaan copied at 43
raw dakwaan copied at 44
raw dakwaan copied at 45
raw dakwaan copied at 46
raw dakwaan copied at 47
raw dakwaan copied at 51
raw dakwaan copied at 52
raw dakwaan copied at 53
raw dakwaan copied at 54
raw dakwaan copied at 55
raw dakwaan copied at 56
raw dakwaan copied at 57
raw dakwaan copied at 58
raw dakwaan copied at 59
raw dakwaan copied at 60
raw dakwaan copied at 61
raw dakwaan copied at 63
raw dakwaan copied at 64
raw dakwaan copied at 65
raw dakwaan copied at 66
raw dak

In [74]:
print(df_copy.iloc[1]['cleaned_dakwaan'])

Dakwaan

 -Bahwa ia, Terdakwa AAN NABAWIYAH bin TARDI pada hari Selasa tanggal 04 Juni 2024 sekitar jam 06.00 WIb atau setidak-tidaknya pada waktu lain dalam bulan Juni 2024 atau setidak-tidaknya pada waktu lain dalam tahun 2024 bertempat di depan Kios PD Pasar Jaya Muara Angke RT.007/011 Kelurahan Pluit Kecamatan Penjaringan Jakarta Utara, atau setidak-tidaknya di suatu tempat yang masih termasuk dalam daerah hukum Pengadilan Negeri Jakarta Utara, mencoba mengambil barang sesuatu, yang seluruhnya atau sebagian kepunyaan orang lain, dengan maksud untuk dimiliki secara melawan hukum, yang dilakukan Terdakwa dengan cara-cara sebagai berikut-
Bahwa pada hari Selasa tanggal 04 Juni 2024 sekira jam 05.30 WIB, terdakwa AAN NABAWIYAH bin TARDI disuruh oleh Sdr. IWAN untuk membeli rokok ke PD Pasar Jaya Muara Angke, lalu terdakwa pergi berjalan kaki dari Kapal KM. ROMSON JAYA 08 yang sandar di Pelabuhan Muara Angke menuju PD Pasar Jaya Muara Angke dan sesampainya di Pasar, terdakwa membeli kop

##### Remove Footer

In [75]:
pattern = r"^(.*?)\s+Jakarta Utara,\s+\d{1,2}\s+\w+\s+\d{4}"

for index, row in df_copy.iterrows():
    text_dakwaan = row['cleaned_dakwaan']
    matches = re.match(pattern, text_dakwaan, re.DOTALL)

    if matches: 
        df_copy.loc[index, "cleaned_dakwaan"] = matches.group(1).strip()
    else:
        df_copy.loc[index, "cleaned_dakwaan"] = text_dakwaan
        print(f"raw dakwaan copied at {index}")


raw dakwaan copied at 0
raw dakwaan copied at 1
raw dakwaan copied at 2
raw dakwaan copied at 3
raw dakwaan copied at 4
raw dakwaan copied at 5
raw dakwaan copied at 6
raw dakwaan copied at 7
raw dakwaan copied at 8
raw dakwaan copied at 9
raw dakwaan copied at 10
raw dakwaan copied at 11
raw dakwaan copied at 12
raw dakwaan copied at 13
raw dakwaan copied at 14
raw dakwaan copied at 15
raw dakwaan copied at 16
raw dakwaan copied at 17
raw dakwaan copied at 18
raw dakwaan copied at 19
raw dakwaan copied at 20
raw dakwaan copied at 21
raw dakwaan copied at 22
raw dakwaan copied at 23
raw dakwaan copied at 24
raw dakwaan copied at 25
raw dakwaan copied at 26
raw dakwaan copied at 27
raw dakwaan copied at 28
raw dakwaan copied at 29
raw dakwaan copied at 30
raw dakwaan copied at 31
raw dakwaan copied at 32
raw dakwaan copied at 33
raw dakwaan copied at 34
raw dakwaan copied at 35
raw dakwaan copied at 36
raw dakwaan copied at 37
raw dakwaan copied at 39
raw dakwaan copied at 40
raw dakwaa

In [76]:
print(df_copy.iloc[1]['cleaned_dakwaan'])

Dakwaan

 -Bahwa ia, Terdakwa AAN NABAWIYAH bin TARDI pada hari Selasa tanggal 04 Juni 2024 sekitar jam 06.00 WIb atau setidak-tidaknya pada waktu lain dalam bulan Juni 2024 atau setidak-tidaknya pada waktu lain dalam tahun 2024 bertempat di depan Kios PD Pasar Jaya Muara Angke RT.007/011 Kelurahan Pluit Kecamatan Penjaringan Jakarta Utara, atau setidak-tidaknya di suatu tempat yang masih termasuk dalam daerah hukum Pengadilan Negeri Jakarta Utara, mencoba mengambil barang sesuatu, yang seluruhnya atau sebagian kepunyaan orang lain, dengan maksud untuk dimiliki secara melawan hukum, yang dilakukan Terdakwa dengan cara-cara sebagai berikut-
Bahwa pada hari Selasa tanggal 04 Juni 2024 sekira jam 05.30 WIB, terdakwa AAN NABAWIYAH bin TARDI disuruh oleh Sdr. IWAN untuk membeli rokok ke PD Pasar Jaya Muara Angke, lalu terdakwa pergi berjalan kaki dari Kapal KM. ROMSON JAYA 08 yang sandar di Pelabuhan Muara Angke menuju PD Pasar Jaya Muara Angke dan sesampainya di Pasar, terdakwa membeli kop

In [77]:
df_copy['cleaned_dakwaan'] = df_copy['cleaned_dakwaan'].str.replace('\n', '. ', regex=False)

In [78]:
print(df_copy.iloc[1]['cleaned_dakwaan'])

Dakwaan. .  -Bahwa ia, Terdakwa AAN NABAWIYAH bin TARDI pada hari Selasa tanggal 04 Juni 2024 sekitar jam 06.00 WIb atau setidak-tidaknya pada waktu lain dalam bulan Juni 2024 atau setidak-tidaknya pada waktu lain dalam tahun 2024 bertempat di depan Kios PD Pasar Jaya Muara Angke RT.007/011 Kelurahan Pluit Kecamatan Penjaringan Jakarta Utara, atau setidak-tidaknya di suatu tempat yang masih termasuk dalam daerah hukum Pengadilan Negeri Jakarta Utara, mencoba mengambil barang sesuatu, yang seluruhnya atau sebagian kepunyaan orang lain, dengan maksud untuk dimiliki secara melawan hukum, yang dilakukan Terdakwa dengan cara-cara sebagai berikut-. Bahwa pada hari Selasa tanggal 04 Juni 2024 sekira jam 05.30 WIB, terdakwa AAN NABAWIYAH bin TARDI disuruh oleh Sdr. IWAN untuk membeli rokok ke PD Pasar Jaya Muara Angke, lalu terdakwa pergi berjalan kaki dari Kapal KM. ROMSON JAYA 08 yang sandar di Pelabuhan Muara Angke menuju PD Pasar Jaya Muara Angke dan sesampainya di Pasar, terdakwa membeli 

In [79]:
df_copy['cleaned_dakwaan'] = df_copy['cleaned_dakwaan'].str.replace('  ', ' ', regex=False)

In [80]:
print(df_copy.iloc[1]['cleaned_dakwaan'])

Dakwaan. . -Bahwa ia, Terdakwa AAN NABAWIYAH bin TARDI pada hari Selasa tanggal 04 Juni 2024 sekitar jam 06.00 WIb atau setidak-tidaknya pada waktu lain dalam bulan Juni 2024 atau setidak-tidaknya pada waktu lain dalam tahun 2024 bertempat di depan Kios PD Pasar Jaya Muara Angke RT.007/011 Kelurahan Pluit Kecamatan Penjaringan Jakarta Utara, atau setidak-tidaknya di suatu tempat yang masih termasuk dalam daerah hukum Pengadilan Negeri Jakarta Utara, mencoba mengambil barang sesuatu, yang seluruhnya atau sebagian kepunyaan orang lain, dengan maksud untuk dimiliki secara melawan hukum, yang dilakukan Terdakwa dengan cara-cara sebagai berikut-. Bahwa pada hari Selasa tanggal 04 Juni 2024 sekira jam 05.30 WIB, terdakwa AAN NABAWIYAH bin TARDI disuruh oleh Sdr. IWAN untuk membeli rokok ke PD Pasar Jaya Muara Angke, lalu terdakwa pergi berjalan kaki dari Kapal KM. ROMSON JAYA 08 yang sandar di Pelabuhan Muara Angke menuju PD Pasar Jaya Muara Angke dan sesampainya di Pasar, terdakwa membeli k

In [81]:
df_copy['cleaned_dakwaan'] = df_copy['cleaned_dakwaan'].str.replace(' .', '.', regex=False)

In [82]:
print(df_copy.iloc[1]['cleaned_dakwaan'])

Dakwaan.. -Bahwa ia, Terdakwa AAN NABAWIYAH bin TARDI pada hari Selasa tanggal 04 Juni 2024 sekitar jam 06.00 WIb atau setidak-tidaknya pada waktu lain dalam bulan Juni 2024 atau setidak-tidaknya pada waktu lain dalam tahun 2024 bertempat di depan Kios PD Pasar Jaya Muara Angke RT.007/011 Kelurahan Pluit Kecamatan Penjaringan Jakarta Utara, atau setidak-tidaknya di suatu tempat yang masih termasuk dalam daerah hukum Pengadilan Negeri Jakarta Utara, mencoba mengambil barang sesuatu, yang seluruhnya atau sebagian kepunyaan orang lain, dengan maksud untuk dimiliki secara melawan hukum, yang dilakukan Terdakwa dengan cara-cara sebagai berikut-. Bahwa pada hari Selasa tanggal 04 Juni 2024 sekira jam 05.30 WIB, terdakwa AAN NABAWIYAH bin TARDI disuruh oleh Sdr. IWAN untuk membeli rokok ke PD Pasar Jaya Muara Angke, lalu terdakwa pergi berjalan kaki dari Kapal KM. ROMSON JAYA 08 yang sandar di Pelabuhan Muara Angke menuju PD Pasar Jaya Muara Angke dan sesampainya di Pasar, terdakwa membeli ko

In [83]:
df_copy.isna().sum()

status_perkara          0
nomor_perkara           0
klasifikasi_perkara     0
tanggal_pendaftaran     0
lama_proses             0
terdakwa                0
penuntut_umum           0
hakim                   0
jumlah_saksi            0
putusan_hukuman         0
barang_bukti            0
dakwaan                 0
cleaned_barang_bukti    0
cleaned_dakwaan         0
dtype: int64

In [84]:
print(f"Jumlah data setelah proses cleaning kolom Dakwaan: {df_copy.shape[0]}")

Jumlah data setelah proses cleaning kolom Dakwaan: 1086


### Save to CSV - STAGE 1 PREPROCESSING

In [85]:
df_copy.to_csv(f'../Data/STAGE 1 PREPROCESSING/STAGE_1_PREPROCESSING_{database_name}_FINAL.csv', index=False)

## Data Transformation

#### Data Transformation: Import Data

In [86]:
df_clean = pd.read_csv(f'../Data/STAGE 1 PREPROCESSING/STAGE_1_PREPROCESSING_{database_name}_FINAL.csv')
df_clean.head(3)

,status_perkara,nomor_perkara,klasifikasi_perkara,tanggal_pendaftaran,lama_proses,terdakwa,penuntut_umum,hakim,jumlah_saksi,putusan_hukuman,barang_bukti,dakwaan,cleaned_barang_bukti,cleaned_dakwaan
0,Minutasi,734/Pid.B/2024/PN Jkt.Utr,Pencurian,"Senin, 19 Agu. 2024",29 Hari,['ALAMSYAH alias ALAM bin AMIR ULUNG'],"['Rachman Rajasa, S.H.', 'Azhary Arsyad Sulaim...",SONTAN MERAUKE SINAGA,2,['ALAMSYAH alias ALAM bin AMIR ULUNG~Pidana Pe...,M E N G A D I L I :\n Menyatakan terd...,c\nDakwaan\n:\n Pertama :\n-----------Bahwa...,berupa 3 (tiga) lembar foto copy STNK mobil bo...,"Dakwaan.. Pertama. -Bahwa ia, Terdakwa ALAMSYA..."
1,Minutasi,712/Pid.B/2024/PN Jkt.Utr,Pencurian,"Selasa, 13 Agu. 2024",35 Hari,['AAN NABAWIYAH BIN TARDI'],"['MELDA SIAGIAN, S.H.', 'ARI SULTON ABDULLAH, ...",TOGI PARDEDE,2,['AAN NABAWIYAH BIN TARDI~Pidana Penjara Waktu...,M E N G A D I L I :\nMenyatakan Ter...,"Dakwaan\n:\n -----------Bahwa ia, Terdakwa ...",1 (satu) unit handpone merek VIVO Type Y27s wa...,"Dakwaan.. -Bahwa ia, Terdakwa AAN NABAWIYAH bi..."
2,Minutasi,707/Pid.B/2024/PN Jkt.Utr,"Penadahan, Penerbitan, dan Pencetakan","Jumat, 09 Agu. 2024",46 Hari,['DAVID ALVIAN bin SYAHRUDIN'],"['TEDDY ANDRI,SH.MH', 'ERMA OCTORA, SH']",DENY RISWANTO,2,['DAVID ALVIAN bin SYAHRUDIN~Pidana Penjara Wa...,MENGADILI:\nMenyatakan Terdakwa DAVID ALVIAN b...,Bahwa Terdakwa DAVID ALVIAN bin SYAHRUDIN pada...,1 (satu) lembar STNK asli motor Honda Beat Nop...,Bahwa Terdakwa DAVID ALVIAN bin SYAHRUDIN pada...


In [87]:
df_clean.isna().sum()

status_perkara          0
nomor_perkara           0
klasifikasi_perkara     0
tanggal_pendaftaran     0
lama_proses             0
terdakwa                0
penuntut_umum           0
hakim                   0
jumlah_saksi            0
putusan_hukuman         0
barang_bukti            0
dakwaan                 0
cleaned_barang_bukti    1
cleaned_dakwaan         0
dtype: int64

In [88]:
null_rows = df_clean[df_clean.isnull().any(axis=1)]
print("Rows with null values:")
print(null_rows)

Rows with null values:
    status_perkara                nomor_perkara klasifikasi_perkara  \
861       Minutasi  993/Pid.Sus/2023/PN Jkt.Utr           Narkotika   

     tanggal_pendaftaran lama_proses  \
861  Senin, 02 Okt. 2023     57 Hari   

                                        terdakwa  \
861  ['AHMAD KORNIAWAN Bin KARNIADI PRIMANTORO']   

                                     penuntut_umum          hakim  \
861  ['ERMA OCTORA, SH', 'PUTU YUMI ANTARI, S.H.']  SLAMET WIDODO   

     jumlah_saksi                                    putusan_hukuman  \
861             2  ['AHMAD KORNIAWAN Bin KARNIADI PRIMANTORO~Subs...   

                                          barang_bukti  \
861  MENGADILI:\nDirampas untuk Negara.\nMembebanka...   

                                               dakwaan cleaned_barang_bukti  \
861  PERTAMA :\n-------------- Bahwa ia Terdakwa AH...                  NaN   

                                       cleaned_dakwaan  
861  PERTAMA. - Bahwa ia Terdak

#### Data Transformation: Drop Null Values

In [89]:
df_clean.dropna(inplace=True)

In [90]:
df_clean.isna().sum()

status_perkara          0
nomor_perkara           0
klasifikasi_perkara     0
tanggal_pendaftaran     0
lama_proses             0
terdakwa                0
penuntut_umum           0
hakim                   0
jumlah_saksi            0
putusan_hukuman         0
barang_bukti            0
dakwaan                 0
cleaned_barang_bukti    0
cleaned_dakwaan         0
dtype: int64

#### Data Transformation: Drop Unused Columns

In [91]:
selected_columns = ['nomor_perkara', 
                    'klasifikasi_perkara', 
                    'terdakwa', 
                    'penuntut_umum', 
                    'hakim', 
                    'jumlah_saksi', 
                    'putusan_hukuman', 
                    'cleaned_barang_bukti', 
                    'cleaned_dakwaan']

df_clean = df_clean[selected_columns].copy()

df_clean.head(3)

,nomor_perkara,klasifikasi_perkara,terdakwa,penuntut_umum,hakim,jumlah_saksi,putusan_hukuman,cleaned_barang_bukti,cleaned_dakwaan
0,734/Pid.B/2024/PN Jkt.Utr,Pencurian,['ALAMSYAH alias ALAM bin AMIR ULUNG'],"['Rachman Rajasa, S.H.', 'Azhary Arsyad Sulaim...",SONTAN MERAUKE SINAGA,2,['ALAMSYAH alias ALAM bin AMIR ULUNG~Pidana Pe...,berupa 3 (tiga) lembar foto copy STNK mobil bo...,"Dakwaan.. Pertama. -Bahwa ia, Terdakwa ALAMSYA..."
1,712/Pid.B/2024/PN Jkt.Utr,Pencurian,['AAN NABAWIYAH BIN TARDI'],"['MELDA SIAGIAN, S.H.', 'ARI SULTON ABDULLAH, ...",TOGI PARDEDE,2,['AAN NABAWIYAH BIN TARDI~Pidana Penjara Waktu...,1 (satu) unit handpone merek VIVO Type Y27s wa...,"Dakwaan.. -Bahwa ia, Terdakwa AAN NABAWIYAH bi..."
2,707/Pid.B/2024/PN Jkt.Utr,"Penadahan, Penerbitan, dan Pencetakan",['DAVID ALVIAN bin SYAHRUDIN'],"['TEDDY ANDRI,SH.MH', 'ERMA OCTORA, SH']",DENY RISWANTO,2,['DAVID ALVIAN bin SYAHRUDIN~Pidana Penjara Wa...,1 (satu) lembar STNK asli motor Honda Beat Nop...,Bahwa Terdakwa DAVID ALVIAN bin SYAHRUDIN pada...


### Data Transformation: Extract Multiple Terdakwa & Putusan Hukuman to each Row 

#### Check Data Type (Expected List)

In [92]:
print(f"Jumlah Data: {len(df_clean)}")

count_str_terdakwa = df_clean[df_clean["terdakwa"].apply(lambda x: isinstance(x, str))].shape[0]
print(f"Jumlah data dengan 'terdakwa' berupa string: {count_str_terdakwa}")

count_str_putusan_hukuman = df_clean[df_clean["putusan_hukuman"].apply(lambda x: isinstance(x, str))].shape[0]
print(f"Jumlah data dengan 'putusan_hukuman' berupa string: {count_str_terdakwa}")

Jumlah Data: 1085
Jumlah data dengan 'terdakwa' berupa string: 1085
Jumlah data dengan 'putusan_hukuman' berupa string: 1085


#### Convert To List

In [93]:
def clean_and_convert_to_list(terdakwa):
    if isinstance(terdakwa, list):
        return terdakwa
    
    if isinstance(terdakwa, str):
        cleaned_str = re.sub(r'\s+', ' ', terdakwa.strip())
        try:

            result_list = ast.literal_eval(cleaned_str)
        except (ValueError, SyntaxError):
            result_list = [cleaned_str]

        if isinstance(result_list, str):
            return [result_list]
        return result_list
    return []

df_clean['terdakwa'] = df_clean['terdakwa'].apply(clean_and_convert_to_list)
df_clean['putusan_hukuman'] = df_clean['putusan_hukuman'].apply(clean_and_convert_to_list)

In [94]:
print(df_clean['terdakwa'].apply(type).unique())
print(df_clean['putusan_hukuman'].apply(type).unique())
print(f"Jumlah Data: {len(df_clean)}")

count_str_terdakwa = df_clean[df_clean["terdakwa"].apply(lambda x: isinstance(x, list))].shape[0]
print(f"Jumlah data dengan 'terdakwa' berupa list: {count_str_terdakwa}")

count_str_putusan_hukuman = df_clean[df_clean["putusan_hukuman"].apply(lambda x: isinstance(x, list))].shape[0]
print(f"Jumlah data dengan 'putusan_hukuman' berupa list: {count_str_terdakwa}")

[<class 'list'>]
[<class 'list'>]
Jumlah Data: 1085
Jumlah data dengan 'terdakwa' berupa list: 1085
Jumlah data dengan 'putusan_hukuman' berupa list: 1085


In [95]:
print(df_clean.loc[18, "putusan_hukuman"])
print(df_clean.loc[18, "terdakwa"])

['MUHAMMAD DALI alias DALZ bin DADANG SUPRIADI~Pidana Penjara Waktu Tertentu (2 Tahun )']
['MUHAMMAD DALI alias DALZ bin DADANG SUPRIADI']


#### Check if Jumlah terdakwa tidak sama dengan Jumlah Putusan (More Data Cleaning)

In [97]:
df_clean['terdakwa_len'] = df_clean['terdakwa'].apply(len)
df_clean['putusan_len'] = df_clean['putusan_hukuman'].apply(len)

mismatched_rows = df_clean[df_clean['terdakwa_len'] != df_clean['putusan_len']]
print(mismatched_rows[['terdakwa_len', 'putusan_len']].head())


     terdakwa_len  putusan_len
14              2            3
27              2            3
74              2            3
148             2            3
240             3            4


In [98]:
print(mismatched_rows)

                     nomor_perkara                 klasifikasi_perkara  \
14     662/Pid.Sus/2024/PN Jkt.Utr                           Narkotika   
27     631/Pid.Sus/2024/PN Jkt.Utr                           Narkotika   
74     569/Pid.Sus/2024/PN Jkt.Utr                           Narkotika   
148    475/Pid.Sus/2024/PN Jkt.Utr                           Narkotika   
240    370/Pid.Sus/2024/PN Jkt.Utr                           Narkotika   
415    170/Pid.Sus/2024/PN Jkt.Utr  Informasi dan Transaksi Elektronik   
611   1310/Pid.Sus/2023/PN Jkt.Utr                           Narkotika   
640   1274/Pid.Sus/2023/PN Jkt.Utr                           Narkotika   
652   1262/Pid.Sus/2023/PN Jkt.Utr                           Narkotika   
727   1170/Pid.Sus/2023/PN Jkt.Utr                           Narkotika   
788   1078/Pid.Sus/2023/PN Jkt.Utr                           Narkotika   
875    987/Pid.Sus/2023/PN Jkt.Utr                           Narkotika   
902    938/Pid.Sus/2023/PN Jkt.Utr    

#### Copy Only if Terdakwa and Putusan is Same 

In [99]:
df_clean = df_clean[df_clean['terdakwa_len'] == df_clean['putusan_len']]
print(df_clean.shape)

(1070, 11)


#### Explode Terdakwa and Putusan

In [100]:
def match_terdakwa_to_putusan(df):
    combined_terdakwa_putusan_list = []
    for index, row in df.iterrows():
        for terdakwa in row['terdakwa']:
            matched = False
            for putusan_hukuman in row['putusan_hukuman']:
                if terdakwa in putusan_hukuman:
                    temp_list = [row['nomor_perkara'], row['klasifikasi_perkara'], row['penuntut_umum'], row['hakim'], row['jumlah_saksi'], row['cleaned_barang_bukti'], row['cleaned_dakwaan'],terdakwa, putusan_hukuman] 
                    combined_terdakwa_putusan_list.append(temp_list)
                    matched = True
                    break
            if not matched:
                print(f'no match {terdakwa} at index {index}')
    df_combined = pd.DataFrame(combined_terdakwa_putusan_list, columns=['nomor_perkara', 'klasifikasi_perkara', 'penuntut_umum', 'hakim', 'jumlah_saksi', 'cleaned_barang_bukti', 'cleaned_dakwaan', 'terdakwa', 'putusan_hukuman'])

    return df_combined

df_clean_exploded = match_terdakwa_to_putusan(df_clean)
print(df_clean_exploded)

                    nomor_perkara                    klasifikasi_perkara  \
0       734/Pid.B/2024/PN Jkt.Utr                              Pencurian   
1       712/Pid.B/2024/PN Jkt.Utr                              Pencurian   
2       707/Pid.B/2024/PN Jkt.Utr  Penadahan, Penerbitan, dan Pencetakan   
3       708/Pid.B/2024/PN Jkt.Utr  Penadahan, Penerbitan, dan Pencetakan   
4       697/Pid.B/2024/PN Jkt.Utr                               Penipuan   
...                           ...                                    ...   
1249  752/Pid.Sus/2023/PN Jkt.Utr                              Narkotika   
1250    738/Pid.B/2023/PN Jkt.Utr                              Pencurian   
1251    738/Pid.B/2023/PN Jkt.Utr                              Pencurian   
1252    740/Pid.B/2023/PN Jkt.Utr                              Pencurian   
1253    740/Pid.B/2023/PN Jkt.Utr                              Pencurian   

                                          penuntut_umum  \
0     ['Rachman Rajasa, S.H.

In [101]:
df_clean_exploded[['nomor_perkara', 'terdakwa', 'putusan_hukuman']].head(5)

,nomor_perkara,terdakwa,putusan_hukuman
0,734/Pid.B/2024/PN Jkt.Utr,ALAMSYAH alias ALAM bin AMIR ULUNG,ALAMSYAH alias ALAM bin AMIR ULUNG~Pidana Penj...
1,712/Pid.B/2024/PN Jkt.Utr,AAN NABAWIYAH BIN TARDI,AAN NABAWIYAH BIN TARDI~Pidana Penjara Waktu T...
2,707/Pid.B/2024/PN Jkt.Utr,DAVID ALVIAN bin SYAHRUDIN,DAVID ALVIAN bin SYAHRUDIN~Pidana Penjara Wakt...
3,708/Pid.B/2024/PN Jkt.Utr,DODIK HERMAWAN BIN ALM. TARJO BIBIT,DODIK HERMAWAN BIN ALM. TARJO BIBIT~Pidana Pen...
4,697/Pid.B/2024/PN Jkt.Utr,ERWIN Bin RADI,ERWIN Bin RADI~Pidana Penjara Waktu Tertentu (...


In [102]:
print(df_clean.shape)
print(df_clean_exploded.shape)
print(df_clean_exploded.isna().sum())

(1070, 11)
(1254, 9)
nomor_perkara           0
klasifikasi_perkara     0
penuntut_umum           0
hakim                   0
jumlah_saksi            0
cleaned_barang_bukti    0
cleaned_dakwaan         0
terdakwa                0
putusan_hukuman         0
dtype: int64


In [103]:
print(len(df_clean_exploded['terdakwa']))
print(len(df_clean_exploded['putusan_hukuman']))

1254
1254


In [104]:
for i in range(10):
    print(i, ".", df_clean_exploded.iloc[i]['putusan_hukuman'])

0 . ALAMSYAH alias ALAM bin AMIR ULUNG~Pidana Penjara Waktu Tertentu (1 Tahun 6 Bulan )
1 . AAN NABAWIYAH BIN TARDI~Pidana Penjara Waktu Tertentu (1 Tahun 2 Bulan )
2 . DAVID ALVIAN bin SYAHRUDIN~Pidana Penjara Waktu Tertentu (10 Bulan )
3 . DODIK HERMAWAN BIN ALM. TARJO BIBIT~Pidana Penjara Waktu Tertentu (10 Bulan )
4 . ERWIN Bin RADI~Pidana Penjara Waktu Tertentu (1 Tahun 8 Bulan )
5 . ALDI HIDAYAT bin ADE HIDAYAT~Pidana Penjara Waktu Tertentu (6 Tahun )
Pidana Denda Rp.1.000.000.000,00
Subsider Penjara (3 Bulan )
6 . HARIYANTO bin alm ALI SODIKIN~Pidana Denda Rp.1.000.000.000,00
Pidana Penjara Waktu Tertentu (7 Tahun 6 Bulan )
Subsider Kurungan (3 Bulan )
7 . WAWAN SUWANDI bin WARTIM~Subsider Penjara (3 Bulan )
Pidana Penjara Waktu Tertentu (6 Tahun )
Pidana Denda Rp.1.000.000.000,00
8 . RAMDAN YUDISTIRA Als YUDIS Bin ROMADON~Pidana Penjara Waktu Tertentu (1 Tahun 4 Bulan )
9 . AHMAD YANI Als BULE~Pidana Penjara Waktu Tertentu (1 Tahun 4 Bulan )


### Data transformation: Extract Multiple Penuntut Umum

#### Check Data Type (Expected List)

In [105]:
unique_types = df_clean_exploded['penuntut_umum'].apply(type).unique()

print(f"Unique data types in the column: {unique_types}")

Unique data types in the column: [<class 'str'>]


#### Convert to List

In [106]:
df_clean_exploded['penuntut_umum'] = df_clean_exploded['penuntut_umum'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

In [107]:
unique_types = df_clean_exploded['penuntut_umum'].apply(type).unique()

print(f"Unique data types in the column: {unique_types}")

Unique data types in the column: [<class 'list'>]


#### Check Penuntut Umum > 1

In [108]:
unique_types = df_clean_exploded['penuntut_umum'].apply(len).unique()

print(f"Unique data types in the column: {unique_types}")

Unique data types in the column: [2 1 3]


In [109]:
list_lengths = df_clean_exploded['penuntut_umum'].apply(len)

unique_lengths = list_lengths.unique()

print(f"Unique lengths of data in the 'penuntut_umum' column: {unique_lengths}")
print("Indices for each unique length:")

for length in unique_lengths:
    indices = list_lengths[list_lengths == length].index
    print(f"Length {length}: Indices - {indices.tolist()}")


Unique lengths of data in the 'penuntut_umum' column: [2 1 3]
Indices for each unique length:
Length 2: Indices - [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 48, 50, 51, 52, 53, 54, 55, 56, 57, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 146, 147, 148, 149, 150, 151, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 213, 214, 

In [110]:
for index, row in df_clean_exploded.iterrows():
    data = row['penuntut_umum']
    if len(data) > 1:
        print(index, row['penuntut_umum'])

0 ['Rachman Rajasa, S.H.', 'Azhary Arsyad Sulaiman']
1 ['MELDA SIAGIAN, S.H.', 'ARI SULTON ABDULLAH, SH']
2 ['TEDDY ANDRI,SH.MH', 'ERMA OCTORA, SH']
3 ['TEDDY ANDRI,SH.MH', 'DANA MAHENDRA, SH']
4 ['DAWIN SOFIAN GAJA, S.H.', 'SADIQA AMALIA, S.H']
5 ['DAWIN SOFIAN GAJA, S.H.', 'BUCHARI TASLIM TUASIKAL, S.H.']
6 ['DAWIN SOFIAN GAJA, S.H.', 'RAKHMAT, SH., MH']
7 ['RAKHMAT, SH., MH', 'DAWIN SOFIAN GAJA, S.H.']
8 ['DHIKI KURNIA, S.H.', 'Rachman Rajasa, S.H.']
9 ['DHIKI KURNIA, S.H.', 'Rachman Rajasa, S.H.']
10 ['DHIKI KURNIA, S.H.', 'Rachman Rajasa, S.H.']
11 ['ZAINAL DWI ARIANTO, SH', 'DONI BOY FAISAL PANJAITAN, SH']
12 ['ZAINAL DWI ARIANTO, SH', 'PUTU YUMI ANTARI, S.H.']
13 ['SADIQA AMALIA, S.H', 'LAWRA RESTI NESYA, S.H.']
14 ['SADIQA AMALIA, S.H', 'DAWIN SOFIAN GAJA, S.H.']
15 ['SHUBHAN NOOR HIDAYAT, SH', 'ARI SULTON ABDULLAH, SH']
16 ['Rachman Rajasa, S.H.', 'DONI BOY FAISAL PANJAITAN, SH']
17 ['Rachman Rajasa, S.H.', 'DONI BOY FAISAL PANJAITAN, SH']
18 ['Rachman Rajasa, S.H.', 'DONI BOY

#### Fix Duplicates Penuntut Umum

In [111]:
# df_clean_exploded['penuntut_umum'] = df_clean_exploded['penuntut_umum'].apply(lambda x: list(set([name.lower() for name in x])))
df_clean_exploded['penuntut_umum'] = df_clean_exploded['penuntut_umum'].apply(lambda x: list(set(x)))
df_clean_exploded['penuntut_umum'] = df_clean_exploded['penuntut_umum'].apply(lambda x: sorted(x))

In [112]:
for index, row in df_clean_exploded.iterrows():
    data = row['penuntut_umum']
    if len(data) > 1:
        print(index, row['penuntut_umum'])

0 ['Azhary Arsyad Sulaiman', 'Rachman Rajasa, S.H.']
1 ['ARI SULTON ABDULLAH, SH', 'MELDA SIAGIAN, S.H.']
2 ['ERMA OCTORA, SH', 'TEDDY ANDRI,SH.MH']
3 ['DANA MAHENDRA, SH', 'TEDDY ANDRI,SH.MH']
4 ['DAWIN SOFIAN GAJA, S.H.', 'SADIQA AMALIA, S.H']
5 ['BUCHARI TASLIM TUASIKAL, S.H.', 'DAWIN SOFIAN GAJA, S.H.']
6 ['DAWIN SOFIAN GAJA, S.H.', 'RAKHMAT, SH., MH']
7 ['DAWIN SOFIAN GAJA, S.H.', 'RAKHMAT, SH., MH']
8 ['DHIKI KURNIA, S.H.', 'Rachman Rajasa, S.H.']
9 ['DHIKI KURNIA, S.H.', 'Rachman Rajasa, S.H.']
10 ['DHIKI KURNIA, S.H.', 'Rachman Rajasa, S.H.']
11 ['DONI BOY FAISAL PANJAITAN, SH', 'ZAINAL DWI ARIANTO, SH']
12 ['PUTU YUMI ANTARI, S.H.', 'ZAINAL DWI ARIANTO, SH']
13 ['LAWRA RESTI NESYA, S.H.', 'SADIQA AMALIA, S.H']
14 ['DAWIN SOFIAN GAJA, S.H.', 'SADIQA AMALIA, S.H']
15 ['ARI SULTON ABDULLAH, SH', 'SHUBHAN NOOR HIDAYAT, SH']
16 ['DONI BOY FAISAL PANJAITAN, SH', 'Rachman Rajasa, S.H.']
17 ['DONI BOY FAISAL PANJAITAN, SH', 'Rachman Rajasa, S.H.']
18 ['DONI BOY FAISAL PANJAITAN, SH', 

#### Explode DataFrame

In [113]:
df_clean_penuntut_exploded = df_clean_exploded.explode('penuntut_umum', ignore_index=True)

In [114]:
df_clean_penuntut_exploded[['nomor_perkara', 'terdakwa', 'penuntut_umum', 'putusan_hukuman']].head(5)

,nomor_perkara,terdakwa,penuntut_umum,putusan_hukuman
0,734/Pid.B/2024/PN Jkt.Utr,ALAMSYAH alias ALAM bin AMIR ULUNG,Azhary Arsyad Sulaiman,ALAMSYAH alias ALAM bin AMIR ULUNG~Pidana Penj...
1,734/Pid.B/2024/PN Jkt.Utr,ALAMSYAH alias ALAM bin AMIR ULUNG,"Rachman Rajasa, S.H.",ALAMSYAH alias ALAM bin AMIR ULUNG~Pidana Penj...
2,712/Pid.B/2024/PN Jkt.Utr,AAN NABAWIYAH BIN TARDI,"ARI SULTON ABDULLAH, SH",AAN NABAWIYAH BIN TARDI~Pidana Penjara Waktu T...
3,712/Pid.B/2024/PN Jkt.Utr,AAN NABAWIYAH BIN TARDI,"MELDA SIAGIAN, S.H.",AAN NABAWIYAH BIN TARDI~Pidana Penjara Waktu T...
4,707/Pid.B/2024/PN Jkt.Utr,DAVID ALVIAN bin SYAHRUDIN,"ERMA OCTORA, SH",DAVID ALVIAN bin SYAHRUDIN~Pidana Penjara Wakt...


In [116]:
print(f"Jumlah data setelah proses transformasi penuntut umum: {df_clean_penuntut_exploded.shape[0]}")

Jumlah data setelah proses transformasi penuntut umum: 2429


### Data Transformation: Extract Lama Kurungan Penjara

In [117]:
pd.set_option('display.max_colwidth', None)
print(df_clean_penuntut_exploded.loc[:10, 'putusan_hukuman'])
pd.reset_option('display.max_colwidth')

0                                                      ALAMSYAH alias ALAM bin AMIR ULUNG~Pidana Penjara Waktu Tertentu (1 Tahun 6 Bulan )
1                                                      ALAMSYAH alias ALAM bin AMIR ULUNG~Pidana Penjara Waktu Tertentu (1 Tahun 6 Bulan )
2                                                                 AAN NABAWIYAH BIN TARDI~Pidana Penjara Waktu Tertentu (1 Tahun 2 Bulan )
3                                                                 AAN NABAWIYAH BIN TARDI~Pidana Penjara Waktu Tertentu (1 Tahun 2 Bulan )
4                                                                     DAVID ALVIAN bin SYAHRUDIN~Pidana Penjara Waktu Tertentu (10 Bulan )
5                                                                     DAVID ALVIAN bin SYAHRUDIN~Pidana Penjara Waktu Tertentu (10 Bulan )
6                                                            DODIK HERMAWAN BIN ALM. TARJO BIBIT~Pidana Penjara Waktu Tertentu (10 Bulan )
7                          

#### Remove Terdakwa Name from Putusan Hukuman

In [118]:
df_clean_penuntut_exploded['cleaned_putusan_hukuman'] = df_clean_penuntut_exploded['putusan_hukuman'].apply(lambda x: x[0].split("~")[1] if isinstance(x, list) and len(x) > 0 else x.split("~")[1])

In [119]:
pd.set_option('display.max_colwidth', None)
print(df_clean_penuntut_exploded.loc[:10, 'cleaned_putusan_hukuman'])
pd.reset_option('display.max_colwidth')

0                                                            Pidana Penjara Waktu Tertentu (1 Tahun 6 Bulan )
1                                                            Pidana Penjara Waktu Tertentu (1 Tahun 6 Bulan )
2                                                            Pidana Penjara Waktu Tertentu (1 Tahun 2 Bulan )
3                                                            Pidana Penjara Waktu Tertentu (1 Tahun 2 Bulan )
4                                                                   Pidana Penjara Waktu Tertentu (10 Bulan )
5                                                                   Pidana Penjara Waktu Tertentu (10 Bulan )
6                                                                   Pidana Penjara Waktu Tertentu (10 Bulan )
7                                                                   Pidana Penjara Waktu Tertentu (10 Bulan )
8                                                            Pidana Penjara Waktu Tertentu (1 Tahun 8 Bulan )
9         

#### Convert to List

In [120]:
df_clean_penuntut_exploded['cleaned_putusan_hukuman'] = df_clean_penuntut_exploded['cleaned_putusan_hukuman'].str.split('\n')

In [121]:
pd.set_option('display.max_colwidth', None)
print(df_clean_penuntut_exploded.loc[:10, 'cleaned_putusan_hukuman'])
pd.reset_option('display.max_colwidth')

0                                                            [Pidana Penjara Waktu Tertentu (1 Tahun 6 Bulan )]
1                                                            [Pidana Penjara Waktu Tertentu (1 Tahun 6 Bulan )]
2                                                            [Pidana Penjara Waktu Tertentu (1 Tahun 2 Bulan )]
3                                                            [Pidana Penjara Waktu Tertentu (1 Tahun 2 Bulan )]
4                                                                   [Pidana Penjara Waktu Tertentu (10 Bulan )]
5                                                                   [Pidana Penjara Waktu Tertentu (10 Bulan )]
6                                                                   [Pidana Penjara Waktu Tertentu (10 Bulan )]
7                                                                   [Pidana Penjara Waktu Tertentu (10 Bulan )]
8                                                            [Pidana Penjara Waktu Tertentu (1 Tahun 8 B

#### Extract Pidana Penjara and Pidana Kurungan Text

In [122]:
pattern = r'Pidana Penjara'
pattern_2 = r'Pidana Kurungan'

for index, row in df_clean_penuntut_exploded.iterrows():
    list_putusan_hukuman = row['cleaned_putusan_hukuman']
    
    for putusan_hukuman in list_putusan_hukuman:
        match = re.search(pattern, putusan_hukuman, re.IGNORECASE)
        match_2 = re.search(pattern_2, putusan_hukuman, re.IGNORECASE)

        if match or match_2:
            df_clean_penuntut_exploded.loc[index, 'extracted_putusan_hukuman'] = putusan_hukuman
            # print(f"{index} -> {putusan_hukuman}")

#### Check Null Extracted Putusan Hukuman

In [123]:
null_values = df_clean_penuntut_exploded[df_clean_penuntut_exploded['extracted_putusan_hukuman'].isna()]
null_values[['nomor_perkara', 'klasifikasi_perkara','cleaned_putusan_hukuman', 'extracted_putusan_hukuman']]

,nomor_perkara,klasifikasi_perkara,cleaned_putusan_hukuman,extracted_putusan_hukuman
1422,1271/Pid.B/2023/PN Jkt.Utr,Penggelapan,[Subsider Penjara (1 Tahun 4 Bulan )],NaN
1423,1271/Pid.B/2023/PN Jkt.Utr,Penggelapan,[Subsider Penjara (1 Tahun 4 Bulan )],NaN
1623,1165/Pid.Sus/2023/PN Jkt.Utr,Narkotika,"[Subsider Kurungan (12 Tahun ), Subsider Denda...",NaN
1624,1165/Pid.Sus/2023/PN Jkt.Utr,Narkotika,"[Subsider Kurungan (12 Tahun ), Subsider Denda...",NaN


#### Use Subsider Penjara to Fill Null Values

In [124]:
pattern = r'Subsider Penjara'
pattern_2 = r'Subsider Kurungan'

for index, row in df_clean_penuntut_exploded.iterrows():
    list_putusan_hukuman = row['cleaned_putusan_hukuman']

    if row['extracted_putusan_hukuman'] is np.nan:
        for putusan_hukuman in list_putusan_hukuman:
            match = re.search(pattern, putusan_hukuman, re.IGNORECASE)
            match_2 = re.search(pattern_2, putusan_hukuman, re.IGNORECASE)

            if match or match_2:
                df_clean_penuntut_exploded.loc[index, 'extracted_putusan_hukuman'] = putusan_hukuman
                # print(f"{index} -> {putusan_hukuman}")
    

In [125]:
df_clean_penuntut_exploded[['nomor_perkara', 'klasifikasi_perkara','cleaned_putusan_hukuman', 'extracted_putusan_hukuman']].head(10)

,nomor_perkara,klasifikasi_perkara,cleaned_putusan_hukuman,extracted_putusan_hukuman
0,734/Pid.B/2024/PN Jkt.Utr,Pencurian,[Pidana Penjara Waktu Tertentu (1 Tahun 6 Bula...,Pidana Penjara Waktu Tertentu (1 Tahun 6 Bulan )
1,734/Pid.B/2024/PN Jkt.Utr,Pencurian,[Pidana Penjara Waktu Tertentu (1 Tahun 6 Bula...,Pidana Penjara Waktu Tertentu (1 Tahun 6 Bulan )
2,712/Pid.B/2024/PN Jkt.Utr,Pencurian,[Pidana Penjara Waktu Tertentu (1 Tahun 2 Bula...,Pidana Penjara Waktu Tertentu (1 Tahun 2 Bulan )
3,712/Pid.B/2024/PN Jkt.Utr,Pencurian,[Pidana Penjara Waktu Tertentu (1 Tahun 2 Bula...,Pidana Penjara Waktu Tertentu (1 Tahun 2 Bulan )
4,707/Pid.B/2024/PN Jkt.Utr,"Penadahan, Penerbitan, dan Pencetakan",[Pidana Penjara Waktu Tertentu (10 Bulan )],Pidana Penjara Waktu Tertentu (10 Bulan )
5,707/Pid.B/2024/PN Jkt.Utr,"Penadahan, Penerbitan, dan Pencetakan",[Pidana Penjara Waktu Tertentu (10 Bulan )],Pidana Penjara Waktu Tertentu (10 Bulan )
6,708/Pid.B/2024/PN Jkt.Utr,"Penadahan, Penerbitan, dan Pencetakan",[Pidana Penjara Waktu Tertentu (10 Bulan )],Pidana Penjara Waktu Tertentu (10 Bulan )
7,708/Pid.B/2024/PN Jkt.Utr,"Penadahan, Penerbitan, dan Pencetakan",[Pidana Penjara Waktu Tertentu (10 Bulan )],Pidana Penjara Waktu Tertentu (10 Bulan )
8,697/Pid.B/2024/PN Jkt.Utr,Penipuan,[Pidana Penjara Waktu Tertentu (1 Tahun 8 Bula...,Pidana Penjara Waktu Tertentu (1 Tahun 8 Bulan )
9,697/Pid.B/2024/PN Jkt.Utr,Penipuan,[Pidana Penjara Waktu Tertentu (1 Tahun 8 Bula...,Pidana Penjara Waktu Tertentu (1 Tahun 8 Bulan )


In [126]:
null_values = df_clean_penuntut_exploded[df_clean_penuntut_exploded['extracted_putusan_hukuman'].isna()]
null_values[['nomor_perkara', 'klasifikasi_perkara','cleaned_putusan_hukuman', 'extracted_putusan_hukuman']]

,nomor_perkara,klasifikasi_perkara,cleaned_putusan_hukuman,extracted_putusan_hukuman


#### Drop Null Values

In [127]:
df_clean_penuntut_exploded.dropna(subset=['extracted_putusan_hukuman'], inplace=True)

In [128]:
df_clean_penuntut_exploded.isna().sum()

nomor_perkara                0
klasifikasi_perkara          0
penuntut_umum                0
hakim                        0
jumlah_saksi                 0
cleaned_barang_bukti         0
cleaned_dakwaan              0
terdakwa                     0
putusan_hukuman              0
cleaned_putusan_hukuman      0
extracted_putusan_hukuman    0
dtype: int64

#### Extract Lama Kurungan Penjara

In [129]:
df_extracted = df_clean_penuntut_exploded.copy()

pattern_tbh = r"\(?\s*(\d+)\s*Tahun\s*(\d+)\s*Bulan\s*(\d+)\s*Hari\s*\)?"
pattern_tb = r"\(?\s*(\d+)\s*Tahun\s*(\d+)\s*Bulan\s*\)?"
pattern_th = r"\(?\s*(\d+)\s*Tahun\s*(\d+)\s*Hari\s*\)?"
pattern_bh = r"\(?\s*(\d+)\s*Bulan\s*(\d+)\s*Hari\s*\)?"
pattern_tahun = r"\s*(\d+)\s*Tahun"
pattern_bulan = r"\s*(\d+)\s*Bulan"
pattern_hari = r"\s*(\d+)\s*Hari"

df_extracted['Tahun'] = None
df_extracted['Bulan'] = None
df_extracted['Hari'] = None

for index, row in df_extracted.iterrows():
    text = row["extracted_putusan_hukuman"]

    match_tbh = re.search(pattern_tbh, text, re.IGNORECASE)
    match_tb = re.search(pattern_tb, text, re.IGNORECASE)
    match_th = re.search(pattern_th, text, re.IGNORECASE)
    match_bh = re.search(pattern_bh, text, re.IGNORECASE)
    match_tahun = re.search(pattern_tahun, text, re.IGNORECASE)
    match_bulan = re.search(pattern_bulan, text, re.IGNORECASE)
    match_hari = re.search(pattern_hari, text, re.IGNORECASE)

    if match_tbh:
        tahun = int(match_tbh.group(1))
        bulan = int(match_tbh.group(2))
        hari = int(match_tbh.group(3))
        print(f"MATCH TBH PATTERN | {text} -> {tahun} Tahun, {bulan} Bulan, {hari} Hari")

    elif match_tb:
        tahun = int(match_tb.group(1))
        bulan = int(match_tb.group(2))
        hari = 0
        print(f"MATCH TB PATTERN | {text} -> {tahun} Tahun, {bulan} Bulan, {hari} Hari")

    elif match_th:
        tahun = int(match_th.group(1))
        bulan = 0
        hari = int(match_th.group(2))
        print(f"MATCH TH PATTERN | {text} -> {tahun} Tahun, {bulan} Bulan, {hari} Hari")

    elif match_bh:
        tahun = 0
        bulan = int(match_bh.group(1))
        hari = int(match_bh.group(2))
        print(f"MATCH BH PATTERN | {text} -> {tahun} Tahun, {bulan} Bulan, {hari} Hari")

    elif match_tahun:
        tahun = int(match_tahun.group(1))
        bulan = 0
        hari = 0
        print(f"MATCH TAHUN PATTERN | {text} -> {tahun} Tahun, {bulan} Bulan, {hari} Hari")

    elif match_bulan:
        tahun = 0
        bulan = int(match_bulan.group(1))
        hari = 0
        print(f"MATCH BULAN PATTERN | {text} -> {tahun} Tahun, {bulan} Bulan, {hari} Hari")

    elif match_hari:
        tahun = 0
        bulan = 0
        hari = int(match_hari.group(1))
        print(f"MATCH HARI PATTERN | {text} -> {tahun} Tahun, {bulan} Bulan, {hari} Hari")

    else:
        tahun = None
        bulan = None
        hari = None

    df_extracted.loc[index, 'Tahun'] = tahun
    df_extracted.loc[index, 'Bulan'] = bulan
    df_extracted.loc[index, 'Hari'] = hari

MATCH TB PATTERN | Pidana Penjara Waktu Tertentu (1 Tahun 6 Bulan ) -> 1 Tahun, 6 Bulan, 0 Hari
MATCH TB PATTERN | Pidana Penjara Waktu Tertentu (1 Tahun 6 Bulan ) -> 1 Tahun, 6 Bulan, 0 Hari
MATCH TB PATTERN | Pidana Penjara Waktu Tertentu (1 Tahun 2 Bulan ) -> 1 Tahun, 2 Bulan, 0 Hari
MATCH TB PATTERN | Pidana Penjara Waktu Tertentu (1 Tahun 2 Bulan ) -> 1 Tahun, 2 Bulan, 0 Hari
MATCH BULAN PATTERN | Pidana Penjara Waktu Tertentu (10 Bulan ) -> 0 Tahun, 10 Bulan, 0 Hari
MATCH BULAN PATTERN | Pidana Penjara Waktu Tertentu (10 Bulan ) -> 0 Tahun, 10 Bulan, 0 Hari
MATCH BULAN PATTERN | Pidana Penjara Waktu Tertentu (10 Bulan ) -> 0 Tahun, 10 Bulan, 0 Hari
MATCH BULAN PATTERN | Pidana Penjara Waktu Tertentu (10 Bulan ) -> 0 Tahun, 10 Bulan, 0 Hari
MATCH TB PATTERN | Pidana Penjara Waktu Tertentu (1 Tahun 8 Bulan ) -> 1 Tahun, 8 Bulan, 0 Hari
MATCH TB PATTERN | Pidana Penjara Waktu Tertentu (1 Tahun 8 Bulan ) -> 1 Tahun, 8 Bulan, 0 Hari
MATCH TAHUN PATTERN | Pidana Penjara Waktu Tertentu 

In [130]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

print(df_extracted[["extracted_putusan_hukuman", "Tahun", "Bulan", "Hari"]])

pd.reset_option('display.max_rows')
pd.reset_option('display.max_colwidth')

                              extracted_putusan_hukuman Tahun Bulan Hari
0      Pidana Penjara Waktu Tertentu (1 Tahun 6 Bulan )     1     6    0
1      Pidana Penjara Waktu Tertentu (1 Tahun 6 Bulan )     1     6    0
2      Pidana Penjara Waktu Tertentu (1 Tahun 2 Bulan )     1     2    0
3      Pidana Penjara Waktu Tertentu (1 Tahun 2 Bulan )     1     2    0
4             Pidana Penjara Waktu Tertentu (10 Bulan )     0    10    0
5             Pidana Penjara Waktu Tertentu (10 Bulan )     0    10    0
6             Pidana Penjara Waktu Tertentu (10 Bulan )     0    10    0
7             Pidana Penjara Waktu Tertentu (10 Bulan )     0    10    0
8      Pidana Penjara Waktu Tertentu (1 Tahun 8 Bulan )     1     8    0
9      Pidana Penjara Waktu Tertentu (1 Tahun 8 Bulan )     1     8    0
10             Pidana Penjara Waktu Tertentu (6 Tahun )     6     0    0
11             Pidana Penjara Waktu Tertentu (6 Tahun )     6     0    0
12     Pidana Penjara Waktu Tertentu (7 Tahun 6 Bul

#### Check Null Values after Extracting Lama Penjara

In [131]:
df_extracted.isnull().any()

nomor_perkara                False
klasifikasi_perkara          False
penuntut_umum                False
hakim                        False
jumlah_saksi                 False
cleaned_barang_bukti         False
cleaned_dakwaan              False
terdakwa                     False
putusan_hukuman              False
cleaned_putusan_hukuman      False
extracted_putusan_hukuman    False
Tahun                        False
Bulan                        False
Hari                         False
dtype: bool

#### Convert Kurungan Penjara menjadi Bulan

In [132]:
df_extracted['total_pidana_penjara_bulan'] = (df_extracted['Tahun'] * 12) + df_extracted['Bulan'] + (df_extracted['Hari'] / 30)

In [133]:
pd.set_option('display.max_rows', None)
print(df_extracted[["Tahun", "Bulan", "Hari", "total_pidana_penjara_bulan"]])
pd.reset_option('display.max_rows')

     Tahun Bulan Hari total_pidana_penjara_bulan
0        1     6    0                       18.0
1        1     6    0                       18.0
2        1     2    0                       14.0
3        1     2    0                       14.0
4        0    10    0                       10.0
5        0    10    0                       10.0
6        0    10    0                       10.0
7        0    10    0                       10.0
8        1     8    0                       20.0
9        1     8    0                       20.0
10       6     0    0                       72.0
11       6     0    0                       72.0
12       7     6    0                       90.0
13       7     6    0                       90.0
14       6     0    0                       72.0
15       6     0    0                       72.0
16       1     4    0                       16.0
17       1     4    0                       16.0
18       1     4    0                       16.0
19       1     4    

In [135]:
print(f"Jumlah data setelah proses ekstraksi label: {df_extracted.shape[0]}")

Jumlah data setelah proses ekstraksi label: 2429


In [134]:
df_extracted.to_csv(f'../Data/STAGE 2 PREPROCESSING/STAGE_2_PREPROCESSING_{database_name}_FINAL.csv', index=False)

## Data Validation

In [136]:
df_stage_1 = pd.read_csv(f'../Data/STAGE 1 PREPROCESSING/STAGE_1_PREPROCESSING_{database_name}_FINAL.csv')
df_stage_2 = pd.read_csv(f'../Data/STAGE 2 PREPROCESSING/STAGE_2_PREPROCESSING_{database_name}_FINAL.csv')

In [137]:
print(df_stage_1.columns)
print(df_stage_2.columns)

Index(['status_perkara', 'nomor_perkara', 'klasifikasi_perkara',
       'tanggal_pendaftaran', 'lama_proses', 'terdakwa', 'penuntut_umum',
       'hakim', 'jumlah_saksi', 'putusan_hukuman', 'barang_bukti', 'dakwaan',
       'cleaned_barang_bukti', 'cleaned_dakwaan'],
      dtype='object')
Index(['nomor_perkara', 'klasifikasi_perkara', 'penuntut_umum', 'hakim',
       'jumlah_saksi', 'cleaned_barang_bukti', 'cleaned_dakwaan', 'terdakwa',
       'putusan_hukuman', 'cleaned_putusan_hukuman',
       'extracted_putusan_hukuman', 'Tahun', 'Bulan', 'Hari',
       'total_pidana_penjara_bulan'],
      dtype='object')


In [138]:
df_validation = df_stage_2[['nomor_perkara', 'klasifikasi_perkara', 'penuntut_umum', 'hakim', 'jumlah_saksi', 'cleaned_barang_bukti', 'cleaned_dakwaan', 'terdakwa', 'total_pidana_penjara_bulan']]

In [139]:
df_validation.head(5)

,nomor_perkara,klasifikasi_perkara,penuntut_umum,hakim,jumlah_saksi,cleaned_barang_bukti,cleaned_dakwaan,terdakwa,total_pidana_penjara_bulan
0,734/Pid.B/2024/PN Jkt.Utr,Pencurian,Azhary Arsyad Sulaiman,SONTAN MERAUKE SINAGA,2,berupa 3 (tiga) lembar foto copy STNK mobil bo...,"Dakwaan.. Pertama. -Bahwa ia, Terdakwa ALAMSYA...",ALAMSYAH alias ALAM bin AMIR ULUNG,18.0
1,734/Pid.B/2024/PN Jkt.Utr,Pencurian,"Rachman Rajasa, S.H.",SONTAN MERAUKE SINAGA,2,berupa 3 (tiga) lembar foto copy STNK mobil bo...,"Dakwaan.. Pertama. -Bahwa ia, Terdakwa ALAMSYA...",ALAMSYAH alias ALAM bin AMIR ULUNG,18.0
2,712/Pid.B/2024/PN Jkt.Utr,Pencurian,"ARI SULTON ABDULLAH, SH",TOGI PARDEDE,2,1 (satu) unit handpone merek VIVO Type Y27s wa...,"Dakwaan.. -Bahwa ia, Terdakwa AAN NABAWIYAH bi...",AAN NABAWIYAH BIN TARDI,14.0
3,712/Pid.B/2024/PN Jkt.Utr,Pencurian,"MELDA SIAGIAN, S.H.",TOGI PARDEDE,2,1 (satu) unit handpone merek VIVO Type Y27s wa...,"Dakwaan.. -Bahwa ia, Terdakwa AAN NABAWIYAH bi...",AAN NABAWIYAH BIN TARDI,14.0
4,707/Pid.B/2024/PN Jkt.Utr,"Penadahan, Penerbitan, dan Pencetakan","ERMA OCTORA, SH",DENY RISWANTO,2,1 (satu) lembar STNK asli motor Honda Beat Nop...,Bahwa Terdakwa DAVID ALVIAN bin SYAHRUDIN pada...,DAVID ALVIAN bin SYAHRUDIN,10.0


#### Validate Data Types

In [140]:
for column in df_validation.columns:
    types_in_column = df_stage_2[column].apply(type).unique()
    print(f"Data types in column '{column}' -> {types_in_column}")

Data types in column 'nomor_perkara' -> [<class 'str'>]
Data types in column 'klasifikasi_perkara' -> [<class 'str'>]
Data types in column 'penuntut_umum' -> [<class 'str'>]
Data types in column 'hakim' -> [<class 'str'>]
Data types in column 'jumlah_saksi' -> [<class 'int'>]
Data types in column 'cleaned_barang_bukti' -> [<class 'str'>]
Data types in column 'cleaned_dakwaan' -> [<class 'str'>]
Data types in column 'terdakwa' -> [<class 'str'>]
Data types in column 'total_pidana_penjara_bulan' -> [<class 'float'>]


#### Merge DataFrame

In [141]:
merged_df = df_validation.merge(df_stage_1, on='nomor_perkara', suffixes=('_stage_2', '_stage_1'))

In [142]:
for column in merged_df.columns:
    types_in_column = merged_df[column].apply(type).unique()
    print(f"Data types in column '{column}' -> {types_in_column}")

Data types in column 'nomor_perkara' -> [<class 'str'>]
Data types in column 'klasifikasi_perkara_stage_2' -> [<class 'str'>]
Data types in column 'penuntut_umum_stage_2' -> [<class 'str'>]
Data types in column 'hakim_stage_2' -> [<class 'str'>]
Data types in column 'jumlah_saksi_stage_2' -> [<class 'int'>]
Data types in column 'cleaned_barang_bukti_stage_2' -> [<class 'str'>]
Data types in column 'cleaned_dakwaan_stage_2' -> [<class 'str'>]
Data types in column 'terdakwa_stage_2' -> [<class 'str'>]
Data types in column 'total_pidana_penjara_bulan' -> [<class 'float'>]
Data types in column 'status_perkara' -> [<class 'str'>]
Data types in column 'klasifikasi_perkara_stage_1' -> [<class 'str'>]
Data types in column 'tanggal_pendaftaran' -> [<class 'str'>]
Data types in column 'lama_proses' -> [<class 'str'>]
Data types in column 'terdakwa_stage_1' -> [<class 'str'>]
Data types in column 'penuntut_umum_stage_1' -> [<class 'str'>]
Data types in column 'hakim_stage_1' -> [<class 'str'>]
Da

In [143]:
merged_df.isna().sum()

nomor_perkara                   0
klasifikasi_perkara_stage_2     0
penuntut_umum_stage_2           0
hakim_stage_2                   0
jumlah_saksi_stage_2            0
cleaned_barang_bukti_stage_2    0
cleaned_dakwaan_stage_2         0
terdakwa_stage_2                0
total_pidana_penjara_bulan      0
status_perkara                  0
klasifikasi_perkara_stage_1     0
tanggal_pendaftaran             0
lama_proses                     0
terdakwa_stage_1                0
penuntut_umum_stage_1           0
hakim_stage_1                   0
jumlah_saksi_stage_1            0
putusan_hukuman                 0
barang_bukti                    0
dakwaan                         0
cleaned_barang_bukti_stage_1    0
cleaned_dakwaan_stage_1         0
dtype: int64

#### Validate Each Row Based on Nomor Perkara (Klasifikasi Perkara, Terdakwa, Penuntut Umum, Hakim, Jumlah Saksi)

##### Klasifikasi Perkara

In [144]:
merged_df['match_klasifikasi_perkara'] = merged_df['klasifikasi_perkara_stage_2'] == merged_df['klasifikasi_perkara_stage_1']

for _, row in merged_df.iterrows():
    if row['match_klasifikasi_perkara']:
        continue
    else:
        print(f"NOMOR PERKARA {row['nomor_perkara']}: KLASIFIKASI PERKARA MISMATCH!")

##### Terdakwa

In [145]:
for index, row in merged_df.iterrows():
    terdakwa_stage_2 = row['terdakwa_stage_2']

    if terdakwa_stage_2 in row['terdakwa_stage_1']:
        continue
    else:
        print(f"NOMOR PERKARA {row['nomor_perkara']}: TERDAKWA MISMATCH!")

##### Penuntut Umum

In [146]:
for index, row in merged_df.iterrows():
    penuntut_umum_stage_2 = row['penuntut_umum_stage_2']

    if penuntut_umum_stage_2 in row['penuntut_umum_stage_1']:
        continue
    else:
        print(f"NOMOR PERKARA {row['nomor_perkara']}: PENUNTUT UMUM MISMATCH!")


##### Hakim

In [147]:
for index, row in merged_df.iterrows():
    hakim_stage_2 = row['hakim_stage_2']

    if hakim_stage_2 in row['hakim_stage_1']:
        continue
    else:
        print(f"NOMOR PERKARA {row['nomor_perkara']}: HAKIM MISMATCH!")

##### Jumlah Saksi

In [148]:
for index, row in merged_df.iterrows():
    if row['jumlah_saksi_stage_2'] == row['jumlah_saksi_stage_1']:
        continue
    else:
        print(f"NOMOR PERKARA {row['nomor_perkara']}: JUMLAH SAKSI MISMATCH!")

#### Drop Unnecessary Columns

In [149]:
merged_df.columns

Index(['nomor_perkara', 'klasifikasi_perkara_stage_2', 'penuntut_umum_stage_2',
       'hakim_stage_2', 'jumlah_saksi_stage_2', 'cleaned_barang_bukti_stage_2',
       'cleaned_dakwaan_stage_2', 'terdakwa_stage_2',
       'total_pidana_penjara_bulan', 'status_perkara',
       'klasifikasi_perkara_stage_1', 'tanggal_pendaftaran', 'lama_proses',
       'terdakwa_stage_1', 'penuntut_umum_stage_1', 'hakim_stage_1',
       'jumlah_saksi_stage_1', 'putusan_hukuman', 'barang_bukti', 'dakwaan',
       'cleaned_barang_bukti_stage_1', 'cleaned_dakwaan_stage_1',
       'match_klasifikasi_perkara'],
      dtype='object')

In [150]:
df_stage_3 = df_validation[['klasifikasi_perkara', 
                            'penuntut_umum', 
                            'hakim', 
                            'jumlah_saksi', 
                            'cleaned_barang_bukti', 
                            'cleaned_dakwaan', 
                            'terdakwa', 
                            'total_pidana_penjara_bulan']]

df_stage_3.to_csv(f'../Data/STAGE 3 PREPROCESSING/STAGE_3_PREPROCESSING_{database_name}_FINAL.csv', index=False)

In [152]:
print(f"Jumlah data setelah proses Data Validation: {df_extracted.shape[0]}")

Jumlah data setelah proses Data Validation: 2429


#### Check Saved CSV

In [151]:
df_stage_3 = pd.read_csv(f'../Data/STAGE 3 PREPROCESSING/STAGE_3_PREPROCESSING_{database_name}_FINAL.csv')

In [153]:
df_stage_3.head(5)

,klasifikasi_perkara,penuntut_umum,hakim,jumlah_saksi,cleaned_barang_bukti,cleaned_dakwaan,terdakwa,total_pidana_penjara_bulan
0,Pencurian,Azhary Arsyad Sulaiman,SONTAN MERAUKE SINAGA,2,berupa 3 (tiga) lembar foto copy STNK mobil bo...,"Dakwaan.. Pertama. -Bahwa ia, Terdakwa ALAMSYA...",ALAMSYAH alias ALAM bin AMIR ULUNG,18.0
1,Pencurian,"Rachman Rajasa, S.H.",SONTAN MERAUKE SINAGA,2,berupa 3 (tiga) lembar foto copy STNK mobil bo...,"Dakwaan.. Pertama. -Bahwa ia, Terdakwa ALAMSYA...",ALAMSYAH alias ALAM bin AMIR ULUNG,18.0
2,Pencurian,"ARI SULTON ABDULLAH, SH",TOGI PARDEDE,2,1 (satu) unit handpone merek VIVO Type Y27s wa...,"Dakwaan.. -Bahwa ia, Terdakwa AAN NABAWIYAH bi...",AAN NABAWIYAH BIN TARDI,14.0
3,Pencurian,"MELDA SIAGIAN, S.H.",TOGI PARDEDE,2,1 (satu) unit handpone merek VIVO Type Y27s wa...,"Dakwaan.. -Bahwa ia, Terdakwa AAN NABAWIYAH bi...",AAN NABAWIYAH BIN TARDI,14.0
4,"Penadahan, Penerbitan, dan Pencetakan","ERMA OCTORA, SH",DENY RISWANTO,2,1 (satu) lembar STNK asli motor Honda Beat Nop...,Bahwa Terdakwa DAVID ALVIAN bin SYAHRUDIN pada...,DAVID ALVIAN bin SYAHRUDIN,10.0


In [154]:
df_stage_3.isna().sum()

klasifikasi_perkara           0
penuntut_umum                 0
hakim                         0
jumlah_saksi                  0
cleaned_barang_bukti          0
cleaned_dakwaan               0
terdakwa                      0
total_pidana_penjara_bulan    0
dtype: int64

# Text Summarization

In [5]:
print("CUDA available:", torch.cuda.is_available())
print("Device count:", torch.cuda.device_count())
print("CUDA version:", torch.version.cuda)

CUDA available: True
Device count: 1
CUDA version: 12.1


In [ ]:
# Check if CUDA is available
if torch.cuda.is_available():
    print(f"Number of CUDA devices: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"Device {i}: {torch.cuda.get_device_name(i)}")
else:
    print("No CUDA devices available.")

In [4]:
import torch
x = torch.rand(5, 3)
print(x)

tensor([[0.1214, 0.0298, 0.0406],
        [0.8481, 0.8833, 0.1258],
        [0.8622, 0.3680, 0.7690],
        [0.6251, 0.0697, 0.8667],
        [0.2934, 0.7982, 0.9171]])


In [15]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Sep_12_02:55:00_Pacific_Daylight_Time_2024
Cuda compilation tools, release 12.6, V12.6.77
Build cuda_12.6.r12.6/compiler.34841621_0


In [16]:
!nvidia-smi

Fri Dec  6 12:56:58 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 566.14                 Driver Version: 566.14         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   49C    P8              4W /   76W |     670MiB /   8188MiB |     13%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Text Summarization "t5-base-indonesian-summarization-cased"

In [5]:
from transformers import pipeline

from rouge_score import rouge_scorer
from nltk.translate.meteor_score import meteor_score
from nltk.translate.bleu_score import sentence_bleu
import torch
from bert_score import score
from nltk.tokenize import word_tokenize
from nltk.translate import meteor_score
import nltk

import time
from torch import float16

nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('punkt')


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Kalea\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Kalea\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Kalea\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### CLEAN \N

In [6]:
df_extracted['cleaned_dakwaan'] = df_extracted['cleaned_dakwaan'].str.replace('\n', ' ', regex=True)

In [9]:
df_extracted['cleaned_dakwaan']

0       DAKWAAN   PERTAMA  - Bahwa Terdakwa KETUT SUBA...
1       DAKWAAN   PERTAMA  - Bahwa Terdakwa KETUT SUBA...
2       DAKWAAN   KESATU             Bahwa ia Terdakwa...
3       DAKWAAN   KESATU                              ...
4       Dakwaan Ke Satu - Bahwa  terdakwa  SANG PUTU W...
                              ...                        
1191    Kesatu  Pasal 83 ayat (1) huruf b UU RI No. 18...
1192    Perbuatan terdakwa sebagaimana diatur dan dian...
1193    Pasal 351 ayat (1) Kitab Undang-undang Hukum p...
1194    Pertama  Pasal 187 ayat (1) KUHP dan Kedua  Pa...
1195          Pasal 362 Kitab Undang-undang Hukum Pidana.
Name: cleaned_dakwaan, Length: 1196, dtype: object

In [ ]:
model_names = [
    "cahya/bert2bert-indonesian-summarization",
    "rowjak/bert-indonesian-news-summarization"
    "cahya/bert2gpt-indonesian-summarization",
    "cahya/t5-base-indonesian-summarization-cased",
    "panggi/t5-small-indonesian-summarization-cased",
    "interstellarx95/mt5-small-finetuned-indonesian-text-summarization-one-epoch",
    "interstellarx95/mt5-small-finetuned-indonesian-text-summarization-v3",
    "rayendito/mt5-small-finetuned-xl-sum-indonesia",
    "interstellarx95/mt5-small-finetuned-indonesian-text-summarization-three-epochs",
]

In [ ]:


pipelines = {
    model_name: pipeline(
        "summarization",
        model=model_name,
        use_fast=True,
        device="cuda",
        batch_size=batch_size,
        torch_dtype="auto"
    )
    for model_name in model_names
}

In [ ]:
import warnings
from transformers import pipeline, AutoTokenizer
pipelines ={}

for model_name in model_names:
    try:
        with warnings.catch_warnings(record=True) as caught_warnings:
            warnings.simplefilter("always")
            tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

            sentencepiece_warning = any(
                "The sentencepiece tokenizer that you are converting to a fast tokenizer" in str(w.message)
                for w in caught_warnings
            )

            if sentencepiece_warning:
                print(f"Warning for {model_name}: Sentencepiece tokenizer fallback detected. Using slow tokenizer.")
                use_fast = False
            else:
                use_fast = True

    except Exception as e:
        print(f"Fast tokenizer not supported for {model_name} due to error: {str(e)}. Using slow tokenizer.")
        use_fast = False

    pipelines[model_name] = pipeline(
        "summarization",
        model=model_name,
        use_fast=use_fast,
        device=0,
    )

Warning for interstellarx95/mt5-small-finetuned-indonesian-text-summarization-one-epoch: Sentencepiece tokenizer fallback detected. Using slow tokenizer.
Warning for interstellarx95/mt5-small-finetuned-indonesian-text-summarization-v3: Sentencepiece tokenizer fallback detected. Using slow tokenizer.


The encoder model config class: <class 'transformers.models.bert.configuration_bert.BertConfig'> is different from the decoder model config class: <class 'transformers.models.gpt2.configuration_gpt2.GPT2Config'>. It is not recommended to use the `AutoTokenizer.from_pretrained()` method in this case. Please use the encoder and decoder specific tokenizer classes.
Config of the encoder: <class 'transformers.models.bert.modeling_bert.BertModel'> is overwritten by shared encoder config: BertConfig {
  "_name_or_path": "cahya/bert-base-indonesian-1.5G",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0

Warning for interstellarx95/mt5-small-finetuned-indonesian-text-summarization-three-epochs: Sentencepiece tokenizer fallback detected. Using slow tokenizer.


Config of the encoder: <class 'transformers.models.bert.modeling_bert.BertModel'> is overwritten by shared encoder config: BertConfig {
  "_name_or_path": "cahya/bert-base-indonesian-1.5G",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.46.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

Config of the decoder: <class 'transformers.models.bert.modeling_bert.BertLMHeadModel'> is overwritten by shared decoder config: BertConfig {
  "_name_or_path": "cahya/bert-base-indonesian-1.5G",
  "add_cross_attent

In [20]:
pipelines

{'cahya/t5-base-indonesian-summarization-cased': <transformers.pipelines.text2text_generation.SummarizationPipeline at 0x243ddf0ece0>,
 'panggi/t5-small-indonesian-summarization-cased': <transformers.pipelines.text2text_generation.SummarizationPipeline at 0x243e05602e0>,
 'interstellarx95/mt5-small-finetuned-indonesian-text-summarization-one-epoch': <transformers.pipelines.text2text_generation.SummarizationPipeline at 0x243d667f610>,
 'interstellarx95/mt5-small-finetuned-indonesian-text-summarization-v3': <transformers.pipelines.text2text_generation.SummarizationPipeline at 0x243e056dae0>,
 'cahya/bert2gpt-indonesian-summarization': <transformers.pipelines.text2text_generation.SummarizationPipeline at 0x243e54cbfd0>,
 'panggi/t5-base-indonesian-summarization-cased': <transformers.pipelines.text2text_generation.SummarizationPipeline at 0x243e54cbe20>,
 'rayendito/mt5-small-finetuned-xl-sum-indonesia': <transformers.pipelines.text2text_generation.SummarizationPipeline at 0x243d38195d0>,


In [17]:
import pprint

pprint.pprint(pipelines)

{'cahya/bert2gpt-indonesian-summarization': <transformers.pipelines.text2text_generation.SummarizationPipeline object at 0x00000243E04B5EA0>,
 'cahya/t5-base-indonesian-summarization-cased': <transformers.pipelines.text2text_generation.SummarizationPipeline object at 0x00000243D3818FA0>,
 'interstellarx95/mt5-small-finetuned-indonesian-text-summarization-one-epoch': <transformers.pipelines.text2text_generation.SummarizationPipeline object at 0x00000243D4E7FA30>,
 'interstellarx95/mt5-small-finetuned-indonesian-text-summarization-three-epochs': <transformers.pipelines.text2text_generation.SummarizationPipeline object at 0x00000243D6723010>,
 'interstellarx95/mt5-small-finetuned-indonesian-text-summarization-v3': <transformers.pipelines.text2text_generation.SummarizationPipeline object at 0x00000243D4E3DCC0>,
 'panggi/t5-base-indonesian-summarization-cased': <transformers.pipelines.text2text_generation.SummarizationPipeline object at 0x00000243D4BB4880>,
 'panggi/t5-small-indonesian-summ

In [8]:
df_extracted

,klasifikasi_perkara,terdakwa,hakim,jumlah_saksi,cleaned_barang_bukti,cleaned_dakwaan,total_pidana_penjara_bulan
0,Kejahatan Perjudian,KETUT SUBAGIA,I Made Bagiarta,3,1 (satu) buah papan bola bergambar 1 (satu) bu...,DAKWAAN PERTAMA - Bahwa Terdakwa KETUT SUBA...,4
1,Kejahatan Perjudian,KOMANG ADITYA,I Made Bagiarta,3,1 (satu) buah papan bola bergambar 1 (satu) bu...,DAKWAAN PERTAMA - Bahwa Terdakwa KETUT SUBA...,4
2,Narkotika,I NYOMAN SUARTA,I Made Bagiarta,3,38 (tiga puluh delapan) potongan pipet plastik...,DAKWAAN KESATU Bahwa ia Terdakwa...,52
3,Narkotika,RICO JAYADI,I Made Bagiarta,3,1 (satu) buah dompet warna hijau 20 (dua puluh...,DAKWAAN KESATU ...,72
4,Narkotika,SANG PUTU WIDIANA,I Gusti Made Juliartawan,4,1(satu) buah HP merk Realme warna hitam putih,Dakwaan Ke Satu - Bahwa terdakwa SANG PUTU W...,48
...,...,...,...,...,...,...,...
1191,Perlindungan dan Pengelolaan Lingkungan Hidup,Jumat Ariyanto,Ida Bagus Bama Dewa. P,3,1 (satu) 1 (satu) batang kayu balang-balang be...,Kesatu Pasal 83 ayat (1) huruf b UU RI No. 18...,12
1192,Kejahatan Perjudian,I KETUT ANA SAPUTRA Alias BENCUT,Sudar,1,36 (tiga puluh enam) lembar kertas rekapan,Perbuatan terdakwa sebagaimana diatur dan dian...,4
1193,Penganiayaan,Sahabudin Alias Udin,Ni Luh Suantini,2,1. 1 (satu) buah tabung gas elpiji ukuran 3 (t...,Pasal 351 ayat (1) Kitab Undang-undang Hukum p...,9
1194,Kejahatan yang Membahayakan Keamananan Umum Ba...,Putu Kristian Damanta,A.A. Sagung Yuni Wulantrisna,4,-1 (satu)unit sepeda motor merk Yamaha Jupiter...,Pertama Pasal 187 ayat (1) KUHP dan Kedua Pa...,24


In [10]:
avg_length = df_extracted['cleaned_dakwaan'].apply(lambda text: len(text.split())).mean()

print(f"Average length of cleaned_dakwaan: {avg_length:.2f} words")

Average length of cleaned_dakwaan: 691.86 words


In [9]:
import datetime
import psutil
import os
def log_memory_usage():
    print(f"CPU usage: {psutil.cpu_percent()}%, Memory usage: {psutil.virtual_memory().percent}%")
    if torch.cuda.is_available():
        print(f"GPU memory allocated: {torch.cuda.memory_allocated() / 1024 ** 2:.2f} MB")
        print(f"GPU memory reserved: {torch.cuda.memory_reserved() / 1024 ** 2:.2f} MB")

save_dir = '../Data/STAGE 3 PREPROCESSING'

if not os.path.exists(save_dir):
    os.makedirs(save_dir)



In [10]:
import warnings
from transformers import pipeline, AutoTokenizer
import time
import datetime
import torch
from torch.amp import autocast

warnings.filterwarnings("ignore", category=UserWarning, module="transformers")
warnings.filterwarnings("ignore", message="Your max_length is set to \d+, but your input_length is only \d+", category=UserWarning, module="transformers")
chunk_size = 450
batch_size = 12

for model_name in model_names:
    pipelines = {}
    try:
        with warnings.catch_warnings(record=True) as caught_warnings:
            warnings.simplefilter("always")  # Capture all warnings
            tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True).to('cuda')

            # Check for the sentencepiece warning
            sentencepiece_warning = any(
                "The sentencepiece tokenizer that you are converting to a fast tokenizer" in str(w.message)
                for w in caught_warnings
            )

            if sentencepiece_warning:
                print(f"Warning for {model_name}: Sentencepiece tokenizer fallback detected. Using slow tokenizer.")
                use_fast = False
            else:
                use_fast = True

    except Exception as e:
        print(f"Fast tokenizer not supported for {model_name} due to error: {str(e)}. Using slow tokenizer.")
        use_fast = False

    # Create the summarization pipeline
    pipelines[model_name] = pipeline(
        "summarization",
        model=model_name,
        use_fast=use_fast,
        device=0,  # Change this to `-1` if not using GPU
    )

    for key, pipe in pipelines.items():
        start_time = time.time()
        summarized_texts = []  # To store summaries for each row

        for index, row in df_extracted.iterrows():
            # print(f"Processing row {index + 1}/{len(df_extracted)} for model {key}")
            if index % 100 == 0:
                print(f"Processing row {index + 1}/{len(df_extracted)} - {index + 100}/{len(df_extracted)} for model {key}")

            text = row['cleaned_dakwaan']

            # Split the text into manageable chunks
            chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

            # Summarize each batch of chunks
            summaries = []
            for i in range(0, len(chunks), batch_size):
                batch = chunks[i:i + batch_size]

                # Dynamically calculate max_length and min_length
                valid_lengths = [len(chunk.split()) for chunk in batch if chunk.strip()]
                if valid_lengths:
                    batch_max_length = min(max(max(valid_lengths) // 3, 100), 200)  # Adjust max range
                    batch_min_length = max(batch_max_length // 2, 50)  # Raise min range
                else:
                    batch_max_length = 100  # Default max for very short chunks
                    batch_min_length = 50  # Default min for very short chunks

                try:
                    with autocast(device_type='cuda'):  # Enable mixed-precision for efficiency
                        results = pipe(
                            batch,
                            max_length=batch_max_length,
                            min_length=batch_min_length,
                            do_sample=False,
                        )
                        summaries.extend([result.get("generated_text", result.get("summary_text", "")) for result in results])
                except Exception as e:
                    print(f"Error with model {key} on batch {i // batch_size}: {e}")
                    summaries.extend(["" for _ in batch])  # Add empty summaries for failed batches

            # Combine all summaries for the current row
            summarized_texts.append(" ".join(summaries))

        # Save summarized texts in the DataFrame
        df_extracted[f'{key}_summarization_dakwaan'] = summarized_texts

        # Save intermediate results to avoid data loss
        timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
        save_dir = f'../Data/STAGE 3 PREPROCESSING/SINGARAJA_TEST_STAGE_3_PREPROCESSING_{timestamp}.csv'
        df_extracted.to_csv(save_dir, index=False)
        print(f"\nCHECKPOINT! Data Saved for model at {save_dir}")

        # Log memory usage (assuming log_memory_usage is implemented elsewhere)
        # log_memory_usage()

        # Free GPU memory
        del pipe
        torch.cuda.empty_cache()
        print(f"Model {key} - Execution time: {time.time() - start_time:.2f} seconds\n")

    # Free pipelines dictionary
    del pipelines
    torch.cuda.empty_cache()


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Fast tokenizer not supported for panggi/t5-base-indonesian-summarization-cased due to error: 'T5TokenizerFast' object has no attribute 'to'. Using slow tokenizer.
Processing row 1/1196 - 100/1196 for model panggi/t5-base-indonesian-summarization-cased


Your max_length is set to 100, but your input_length is only 85. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=42)
Your max_length is set to 100, but your input_length is only 90. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=45)
Your max_length is set to 100, but your input_length is only 90. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=45)
Your max_length is set to 100, but your input_length is only 93. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=46)
Your

Processing row 101/1196 - 200/1196 for model panggi/t5-base-indonesian-summarization-cased


Your max_length is set to 100, but your input_length is only 85. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=42)
Your max_length is set to 100, but your input_length is only 88. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=44)
Your max_length is set to 100, but your input_length is only 99. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=49)
Your max_length is set to 100, but your input_length is only 93. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=46)
Your

...

Processing row 1001/1196 - 1100/1196 for model panggi/t5-base-indonesian-summarization-cased


Your max_length is set to 100, but your input_length is only 99. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=49)
Your max_length is set to 100, but your input_length is only 9. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=4)
Your max_length is set to 100, but your input_length is only 94. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=47)
Your max_length is set to 100, but your input_length is only 84. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=42)
Your m

Processing row 1101/1196 - 1200/1196 for model panggi/t5-base-indonesian-summarization-cased


Your max_length is set to 100, but your input_length is only 94. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=47)
Your max_length is set to 100, but your input_length is only 79. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=39)
Your max_length is set to 100, but your input_length is only 90. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=45)
Your max_length is set to 100, but your input_length is only 89. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=44)
Your


CHECKPOINT! Data Saved for model at ../Data/STAGE 3 PREPROCESSING/SINGARAJA_TEST_STAGE_3_PREPROCESSING_20241208_055637.csv
Model panggi/t5-base-indonesian-summarization-cased - Execution time: 20091.55 seconds



In [100]:
df_tempor = pd.read_csv('../Data/STAGE 3 PREPROCESSING/Final Singaraja Summarization.csv')

In [108]:
df_tempor.columns

Index(['Unnamed: 0', 'klasifikasi_perkara', 'terdakwa', 'hakim',
       'jumlah_saksi', 'cleaned_barang_bukti', 'cleaned_dakwaan',
       'total_pidana_penjara_bulan',
       'panggi/t5-small-indonesian-summarization-cased_summarization_dakwaan',
       'interstellarx95/mt5-small-finetuned-indonesian-text-summarization-one-epoch_summarization_dakwaan',
       'interstellarx95/mt5-small-finetuned-indonesian-text-summarization-v3_summarization_dakwaan',
       'rayendito/mt5-small-finetuned-xl-sum-indonesia_summarization_dakwaan',
       'interstellarx95/mt5-small-finetuned-indonesian-text-summarization-three-epochs_summarization_dakwaan',
       'panggi/t5-base-indonesian-summarization-cased_summarization_dakwaan',
       'cahya_bert2bert_summarization_dakwaan',
       'rowjak/bert-indonesian-news-summarization',
       'cahya_bert2gpt_summarization_dakwaan',
       'cahya/t5-base-indonesian-summarization-cased_summarization_dakwaan'],
      dtype='object')

In [107]:
df_tempor.iloc[:, -12:]

,cleaned_dakwaan,total_pidana_penjara_bulan,panggi/t5-small-indonesian-summarization-cased_summarization_dakwaan,interstellarx95/mt5-small-finetuned-indonesian-text-summarization-one-epoch_summarization_dakwaan,interstellarx95/mt5-small-finetuned-indonesian-text-summarization-v3_summarization_dakwaan,rayendito/mt5-small-finetuned-xl-sum-indonesia_summarization_dakwaan,interstellarx95/mt5-small-finetuned-indonesian-text-summarization-three-epochs_summarization_dakwaan,panggi/t5-base-indonesian-summarization-cased_summarization_dakwaan,cahya_bert2bert_summarization_dakwaan,rowjak/bert-indonesian-news-summarization,cahya_bert2gpt_summarization_dakwaan,cahya/t5-base-indonesian-summarization-cased_summarization_dakwaan
0,DAKWAAN PERTAMA - Bahwa Terdakwa KETUT SUBA...,4,Bahwa Terdakwa KETUT SUBAGIA bersama dengan te...,KeTUT SUBAGIA bersama dengan Terdakwa KOMANG A...,Bahwa Terdakwa KETUT SUBAGIA bersama dengan Te...,Seorang terdakwa kecelakaan pertama - telah me...,Bahwa Terdakwa KETUT SUBAGIA bersama dengan Te...,Pemain permainan judi bola adil b...,dakwaan pertama - kedua terdakwa ketut subagia...,dakwaan pertama - kedua terdakwa ketut subagia...,dakwaan pertama - bahwa terdakwa ketut subagia...,DAKWAAN PERTAMA- Bahwa Terdakwa KETUT SUBAGIA ...
1,DAKWAAN PERTAMA - Bahwa Terdakwa KETUT SUBA...,4,Bahwa Terdakwa KETUT SUBAGIA bersama dengan te...,KeTUT SUBAGIA bersama dengan Terdakwa KOMANG A...,Bahwa Terdakwa KETUT SUBAGIA bersama dengan Te...,Seorang terdakwa kecelakaan pertama - telah me...,Bahwa Terdakwa KETUT SUBAGIA bersama dengan Te...,Pemain permainan judi bola adil b...,dakwaan pertama - kedua terdakwa ketut subagia...,dakwaan pertama - kedua terdakwa ketut subagia...,dakwaan pertama - bahwa terdakwa ketut subuku ...,DAKWAAN PERTAMA- Bahwa Terdakwa KETUT SUBAGIA ...
2,DAKWAAN KESATU Bahwa ia Terdakwa...,52,DAKWAAN KESATU Bahwa ia Terdakwa I NYOMAN SUAR...,Kepolisian KESATU Bahwa ia Terdakwa I NYOMAN S...,DAKWAAN KESATU Bahwa ia Terdakwa I NYOMAN SUAR...,Seorang pria terdakwa seorang pria yang terdak...,DKWAAN KESATU Bahwa ia Terdakwa I NYOMAN SUART...,Bahwa selanjutnya dilakukan penimban...,dakwaan kesatu bahwa ia terdakwa i nyoman suar...,dakwaan kesatu bahwa ia terdakwa i nyoman suar...,dakwaan kesatu bahwa terdakwa i nyoman suarta ...,DAKWAAN KESATU Bahwa ia Terdakwa I NYOMAN SUAR...
3,DAKWAAN KESATU ...,72,DAKWAAN KESATU Bahwa ia Terdakwa RICO JAYADI p...,Kedakwaan KESATU Bahwa ia Terdakwa RICO JAYADI...,DAKWAAN KESATU Bahwa ia Terdakwa RICO JAYADI p...,"Seorang warga Banjar, RICOJAYADI, terdakwa RIC...",DKWAAN KESATU Bahwa ia Terdakwa RICO JAYADI pa...,Plastik klip berisi butiran kristal ...,dakwaan kesatu bahwa ia terdakwa rico jayadi p...,dakwaan kesatu bahwa ia terdakwa rico jayadi p...,dakwaan kesatu bahwa ia terdakwa rico jayadi p...,DAKWAAN KESATU Bahwa ia Terdakwa RICO JAYADI p...
4,Dakwaan Ke Satu - Bahwa terdakwa SANG PUTU W...,48,Dakwaan Ke Satu- Bahwa terdakwa SANG PUTU WIDI...,Ke Satu - Bahwa terdakwa SANG PUTU WIDIANA ber...,Dakwaan Ke Satu - Bahwa terdakwa SANG PUTU WID...,"Seorang terdakwa SANG PUTU WIDIANA, seorang sa...",Dakwaan Ke Satu - Bahwa terdakwa SANG PUTU WID...,Bahwa terhadap barang bukti berupa 6...,dakwaan ke satu - satunya tentang terdakwa san...,dakwaan ke satu - satunya ditujukan terhadap t...,dakwaan terhadap terdakwa sang putu widiana be...,Dakwaan Ke Satu- Bahwa terdakwa SANG PUTU WIDI...
...,...,...,...,...,...,...,...,...,...,...,...,...
1191,Kesatu Pasal 83 ayat (1) huruf b UU RI No. 18...,12,Kesatu Pasal 83 ayat ( 1 ) huruf b UU RI No. 1...,Kesatu Pasal 83 ayat (1) huruf b UU RI No. 18 ...,Kesatu Pasal 83 ayat 1 huruf b UU RI No. 18 ta...,Sebuah Pasal 83 ayat 1 huruf b UU RI No. 18 ta...,Kesatu Pasal 83 ayat (1) huruf b UU RI No. 18 ...,NaN,kesatu pasal 83 ayat ( 1 ) huruf b uu ri no. 1...,dua pasal dalam uu ri nomor 18 tahun 2013 tent...,kesatu pasal 83 ayat ( 1 ) huruf b undang - un...,Kesatu Pasal 83 ayat(1) huruf b UU RI No. 18 t...
1192,Perbuatan terdakwa sebagaimana diatur da

# single model


In [ ]:
import warnings
from transformers import BertTokenizer, EncoderDecoderModel
import time
import torch
import datetime
from torch.amp import autocast

# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained("cahya/bert2bert-indonesian-summarization")
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token
model = EncoderDecoderModel.from_pretrained("cahya/bert2bert-indonesian-summarization").to('cuda')

# Parameters
chunk_size = 450
batch_size = 16  # Number of chunks per batch for model inference

# To store summaries
summarized_texts = []

start_time = time.time()

# Iterate over rows
for index, row in df_extracted.iterrows():
    if index % 100 == 0:
        print(f"Processing row {index + 1}/{len(df_extracted)} - {index + 100}/{len(df_extracted)}")
    text = row['cleaned_dakwaan']

    # Split the text into manageable chunks
    chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

    # Summarize each batch of chunks
    summaries = []
    for i in range(0, len(chunks), batch_size):
        batch = chunks[i:i + batch_size]

        # Tokenize the batch
        encoded_input = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=chunk_size).to('cuda')

        # Dynamic length adjustment
        valid_lengths = [len(chunk.split()) for chunk in batch if chunk.strip()]
        if valid_lengths:
            batch_max_length = min(max(max(valid_lengths) // 3, 100), 200)  # Adjust max range
            batch_min_length = max(batch_max_length // 2, 50)  # Raise min range
        else:
            batch_max_length = 100  # Default max for very short chunks
            batch_min_length = 50  # Default min for very short chunks

        try:
            with autocast(device_type='cuda'):
            # Generate summaries
                summary_ids = model.generate(
                    input_ids=encoded_input['input_ids'],
                    attention_mask=encoded_input['attention_mask'],  # Include this line
                    min_length=batch_min_length,
                    max_length=batch_max_length,
                    num_beams=10,
                    repetition_penalty=2.5,
                    length_penalty=1.0,
                    early_stopping=True,
                    no_repeat_ngram_size=2,
                    use_cache=True,
                    do_sample=True,
                    temperature=0.8,
                    top_k=50,
                    top_p=0.95
                )
                summaries.extend([tokenizer.decode(ids, skip_special_tokens=True) for ids in summary_ids])
        except Exception as e:
            print(f"Error processing batch {i // batch_size}: {e}")
            summaries.extend(["" for _ in batch])  # Add empty summaries for failed batches

    # Combine all summaries for the current row
    # print(summaries)
    summarized_texts.append(" ".join(summaries))
    del encoded_input, summary_ids
    torch.cuda.empty_cache()

# Save summarized texts in the DataFrame
df_extracted['cahya_bert2bert_summarization_dakwaan'] = summarized_texts

# Save intermediate results to avoid data loss
timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
save_dir = f'../Data/STAGE 3 PREPROCESSING/SINGARAJA_TEST_STAGE_3_PREPROCESSING_{timestamp}.csv'
df_extracted.to_csv(save_dir, index=False)
print(f"\nCHECKPOINT! Data Saved for model {model_name} at {save_dir}")
print(f"Summarization completed in {time.time() - start_time:.2f} seconds")



Config of the encoder: <class 'transformers.models.bert.modeling_bert.BertModel'> is overwritten by shared encoder config: BertConfig {
  "_name_or_path": "cahya/bert-base-indonesian-1.5G",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.46.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

Config of the decoder: <class 'transformers.models.bert.modeling_bert.BertLMHeadModel'> is overwritten by shared decoder config: BertConfig {
  "_name_or_path": "cahya/bert-base-indonesian-1.5G",
  "add_cross_attent

Processing row 1/1196 - 100/1196


We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Processing row 101/1196 - 200/1196
Processing row 201/1196 - 300/1196
Processing row 301/1196 - 400/1196
Processing row 401/1196 - 500/1196
Processing row 501/1196 - 600/1196
Processing row 601/1196 - 700/1196
Processing row 701/1196 - 800/1196
Processing row 801/1196 - 900/1196
Processing row 901/1196 - 1000/1196
Processing row 1001/1196 - 1100/1196
Processing row 1101/1196 - 1200/1196


OSError: Cannot save file into a non-existent directory: '..\Data\STAGE_3_PREPROCESSING'

In [12]:
# Save intermediate results to avoid data loss
timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
save_dir = f'../Data/STAGE 3 PREPROCESSING/SINGARAJA_TEST_STAGE_3_PREPROCESSING_{timestamp}.csv'
df_extracted.to_csv(save_dir, index=False)
print(f"\nCHECKPOINT! Data Saved for model cahya/bert2bert at {save_dir}")
print(f"Summarization completed in {time.time() - start_time:.2f} seconds")


CHECKPOINT! Data Saved for model cahya/bert2bert at ../Data/STAGE 3 PREPROCESSING/SINGARAJA_TEST_STAGE_3_PREPROCESSING_20241207_101016.csv
Summarization completed in 8207.40 seconds


In [ ]:
import warnings
from transformers import BertTokenizer, EncoderDecoderModel
import time
import torch
import datetime
from torch.amp import autocast

# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained("rowjak/bert-indonesian-news-summarization")
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token
model = EncoderDecoderModel.from_pretrained("rowjak/bert-indonesian-news-summarization").to('cuda')

# Parameters
chunk_size = 450
batch_size = 32  # Number of chunks per batch for model inference

# To store summaries
summarized_texts = []

start_time = time.time()

# Iterate over rows
for index, row in df_extracted.iterrows():
    if index % 100 == 0:
        print(f"Processing row {index + 1}/{len(df_extracted)} - {index + 100}/{len(df_extracted)}")
    text = row['cleaned_dakwaan']

    # Split the text into manageable chunks
    chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

    # Summarize each batch of chunks
    summaries = []
    for i in range(0, len(chunks), batch_size):
        batch = chunks[i:i + batch_size]

        # Tokenize the batch
        encoded_input = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=chunk_size).to('cuda')

        # Dynamic length adjustment
        valid_lengths = [len(chunk.split()) for chunk in batch if chunk.strip()]
        if valid_lengths:
            batch_max_length = min(max(max(valid_lengths) // 3, 100), 200)  # Adjust max range
            batch_min_length = max(batch_max_length // 2, 50)  # Raise min range
        else:
            batch_max_length = 100  # Default max for very short chunks
            batch_min_length = 50  # Default min for very short chunks

        try:
            with autocast(device_type='cuda'):
            # Generate summaries
                summary_ids = model.generate(
                    input_ids=encoded_input['input_ids'],
                    attention_mask=encoded_input['attention_mask'],
                    num_beams=2,
                    repetition_penalty=2.5, 
                    length_penalty=1.0, 
                    early_stopping=True,
                    no_repeat_ngram_size=2,
                    use_cache=True,
                    max_length=batch_max_length,
                    min_length=batch_min_length)

                summaries.extend([tokenizer.decode(ids, skip_special_tokens=True) for ids in summary_ids])
        except Exception as e:
            print(f"Error processing batch {i // batch_size}: {e}")
            summaries.extend(["" for _ in batch])  # Add empty summaries for failed batches

    # Combine all summaries for the current row
    # print(summaries)
    summarized_texts.append(" ".join(summaries))
    del encoded_input, summary_ids
    torch.cuda.empty_cache()

# Save summarized texts in the DataFrame
df_extracted['rowjak/bert-indonesian-news-summarization'] = summarized_texts

# Save intermediate results to avoid data loss
timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
save_dir = f'../Data/STAGE 3 PREPROCESSING/SINGARAJA_TEST_STAGE_3_PREPROCESSING_{timestamp}.csv'
df_extracted.to_csv(save_dir, index=False)
print(f"\nCHECKPOINT! Data Saved for model at {save_dir}")
print(f"Summarization completed in {time.time() - start_time:.2f} seconds")



Config of the encoder: <class 'transformers.models.bert.modeling_bert.BertModel'> is overwritten by shared encoder config: BertConfig {
  "_name_or_path": "cahya/bert-base-indonesian-1.5G",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.46.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

Config of the decoder: <class 'transformers.models.bert.modeling_bert.BertLMHeadModel'> is overwritten by shared decoder config: BertConfig {
  "_name_or_path": "cahya/bert-base-indonesian-1.5G",
  "add_cross_attent

Processing row 1/1196 - 100/1196
Processing row 101/1196 - 200/1196
Processing row 201/1196 - 300/1196
Processing row 301/1196 - 400/1196
Processing row 401/1196 - 500/1196
Processing row 501/1196 - 600/1196
Processing row 601/1196 - 700/1196
Processing row 701/1196 - 800/1196
Processing row 801/1196 - 900/1196
Processing row 901/1196 - 1000/1196
Processing row 1001/1196 - 1100/1196
Processing row 1101/1196 - 1200/1196


NameError: name 'model_name' is not defined

In [14]:
timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
save_dir = f'../Data/STAGE 3 PREPROCESSING/SINGARAJA_TEST_STAGE_3_PREPROCESSING_{timestamp}.csv'
df_extracted.to_csv(save_dir, index=False)
print(f"\nCHECKPOINT! Data Saved for model at {save_dir}")


CHECKPOINT! Data Saved for model at ../Data/STAGE 3 PREPROCESSING/SINGARAJA_TEST_STAGE_3_PREPROCESSING_20241207_105235.csv


In [15]:
import warnings
from transformers import BertTokenizer, EncoderDecoderModel
import time
import torch
import datetime
from torch.amp import autocast

# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained("cahya/bert2gpt-indonesian-summarization")
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token
model = EncoderDecoderModel.from_pretrained("cahya/bert2gpt-indonesian-summarization").to('cuda')

# Parameters
chunk_size = 450
batch_size = 32  # Number of chunks per batch for model inference

# To store summaries
summarized_texts = []

start_time = time.time()

# Iterate over rows
for index, row in df_extracted.iterrows():
    if index % 100 == 0:
        print(f"Processing row {index + 1}/{len(df_extracted)} - {index + 100}/{len(df_extracted)}")
    text = row['cleaned_dakwaan']

    # Split the text into manageable chunks
    chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

    # Summarize each batch of chunks
    summaries = []
    for i in range(0, len(chunks), batch_size):
        batch = chunks[i:i + batch_size]

        # Tokenize the batch
        encoded_input = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=chunk_size).to('cuda')

        # Dynamic length adjustment
        valid_lengths = [len(chunk.split()) for chunk in batch if chunk.strip()]
        if valid_lengths:
            batch_max_length = min(max(max(valid_lengths) // 3, 100), 200)  # Adjust max range
            batch_min_length = max(batch_max_length // 2, 50)  # Raise min range
        else:
            batch_max_length = 100  # Default max for very short chunks
            batch_min_length = 50  # Default min for very short chunks

        try:
            with autocast(device_type='cuda'):
            # Generate summaries
                summary_ids = model.generate(
                    input_ids=encoded_input['input_ids'],
                    attention_mask=encoded_input['attention_mask'],  # Include this line
                    min_length=batch_min_length,
                    max_length=batch_max_length,
                    num_beams=10,
                    repetition_penalty=2.5,
                    length_penalty=1.0,
                    early_stopping=True,
                    no_repeat_ngram_size=2,
                    use_cache=True,
                    do_sample=True,
                    temperature=0.8,
                    top_k=50,
                    top_p=0.95
                )
                summaries.extend([tokenizer.decode(ids, skip_special_tokens=True) for ids in summary_ids])
        except Exception as e:
            print(f"Error processing batch {i // batch_size}: {e}")
            summaries.extend(["" for _ in batch])  # Add empty summaries for failed batches

    # Combine all summaries for the current row
    # print(summaries)
    summarized_texts.append(" ".join(summaries))
    del encoded_input, summary_ids
    torch.cuda.empty_cache()

# Save summarized texts in the DataFrame
df_extracted['cahya_bert2gpt_summarization_dakwaan'] = summarized_texts

# Save intermediate results to avoid data loss
timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
save_dir = f'../Data/STAGE 3 PREPROCESSING/SINGARAJA_TEST_STAGE_3_PREPROCESSING_{timestamp}.csv'
df_extracted.to_csv(save_dir, index=False)
print(f"\nCHECKPOINT! Data Saved for model at {save_dir}")
print(f"Summarization completed in {time.time() - start_time:.2f} seconds")



Config of the encoder: <class 'transformers.models.bert.modeling_bert.BertModel'> is overwritten by shared encoder config: BertConfig {
  "_name_or_path": "cahya/bert-base-indonesian-1.5G",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.46.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

Config of the decoder: <class 'transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel'> is overwritten by shared decoder config: GPT2Config {
  "_name_or_path": "cahya/gpt2-small-indonesian-522M",
  "activation_func

Processing row 1/1196 - 100/1196
Processing row 101/1196 - 200/1196
Processing row 201/1196 - 300/1196
Processing row 301/1196 - 400/1196
Processing row 401/1196 - 500/1196
Processing row 501/1196 - 600/1196
Processing row 601/1196 - 700/1196
Processing row 701/1196 - 800/1196
Processing row 801/1196 - 900/1196
Processing row 901/1196 - 1000/1196
Processing row 1001/1196 - 1100/1196
Processing row 1101/1196 - 1200/1196

CHECKPOINT! Data Saved for model at ../Data/STAGE 3 PREPROCESSING/SINGARAJA_TEST_STAGE_3_PREPROCESSING_20241207_122142.csv
Summarization completed in 5332.07 seconds


In [27]:
del encoded_input
del summary_ids
del model
del tokenizer
torch.cuda.empty_cache()

NameError: name 'summary_ids' is not defined

In [ ]:
# dont run!

import warnings
from transformers import pipeline, AutoTokenizer
import time
import datetime
import torch
from torch.amp import autocast

chunk_size = 450
batch_size = 2048

for model_name in model_names:
    break
    pipelines = {}
    try:
        with warnings.catch_warnings(record=True) as caught_warnings:
            warnings.simplefilter("always")  # Capture all warnings
            tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

            # Check for the sentencepiece warning
            sentencepiece_warning = any(
                "The sentencepiece tokenizer that you are converting to a fast tokenizer" in str(w.message)
                for w in caught_warnings
            )

            if sentencepiece_warning:
                print(f"Warning for {model_name}: Sentencepiece tokenizer fallback detected. Using slow tokenizer.")
                use_fast = False
            else:
                use_fast = True

    except Exception as e:
        print(f"Fast tokenizer not supported for {model_name} due to error: {str(e)}. Using slow tokenizer.")
        use_fast = False

    # Create the summarization pipeline
    pipelines[model_name] = pipeline(
        "summarization",
        model=model_name,
        use_fast=use_fast,
        device=0,  # Change this to `-1` if not using GPU
    )

    for key, pipe in pipelines.items():
        start_time = time.time()
        summarized_texts = []  # To store summaries for each row

        for index, row in df_extracted.iterrows():
            # Print progress every 100 rows
            print(f"Processing row {index + 1}/{len(df_extracted)} for model {key}")
            # if index % 50 == 0:
                # print(f"Processing row {index + 1}/{len(df_extracted)} - {index + 50}/{len(df_extracted)} for model {key}")

            text = row['cleaned_dakwaan']

            # Split the text into manageable chunks
            chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

            # Summarize each batch of chunks
            summaries = []
            for i in range(0, len(chunks), batch_size):
                batch = chunks[i:i + batch_size]

                # Dynamically calculate max_length and min_length
                valid_lengths = [len(chunk.split()) for chunk in batch if chunk.strip()]
                if valid_lengths:
                    batch_max_length = max(min(max(length // 3 for length in valid_lengths), 150), 30)
                    batch_min_length = max(batch_max_length // 2, 20)
                else:
                    batch_max_length = 30
                    batch_min_length = 20

                try:
                    with autocast(device_type='cuda'):  # Enable mixed-precision for efficiency
                        results = pipe(
                            batch,
                            max_length=batch_max_length,
                            min_length=batch_min_length,
                            do_sample=False,
                        )
                        summaries.extend([result.get("generated_text", result.get("summary_text", "")) for result in results])
                except Exception as e:
                    print(f"Error with model {key} on batch {i // batch_size}: {e}")
                    summaries.extend(["" for _ in batch])  # Add empty summaries for failed batches

            # Combine all summaries for the current row
            summarized_texts.append(" ".join(summaries))

        # Save summarized texts in the DataFrame
        df_extracted[f'{key}_summarization_dakwaan'] = summarized_texts

        # Save intermediate results to avoid data loss
        timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
        save_dir = f'../Data/STAGE_3_PREPROCESSING/SINGARAJA_TEST_STAGE_3_PREPROCESSING_{timestamp}.csv'
        df_extracted.to_csv(save_dir, index=False)
        print(f"\nCHECKPOINT! Data Saved for model {key} at {save_dir}")

        # Log memory usage (assuming log_memory_usage is implemented elsewhere)
        # log_memory_usage()

        # Free GPU memory
        del pipes
        torch.cuda.empty_cache()
        print(f"Model {key} - Execution time: {time.time() - start_time:.2f} seconds\n")

    # Free pipelines dictionary
    del pipelines
    torch.cuda.empty_cache()


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Processing row 1/1196 for model cahya/t5-base-indonesian-summarization-cased
Processing row 2/1196 for model cahya/t5-base-indonesian-summarization-cased
Processing row 3/1196 for model cahya/t5-base-indonesian-summarization-cased


Your max_length is set to 30, but your input_length is only 9. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=4)


Processing row 4/1196 for model cahya/t5-base-indonesian-summarization-cased


Your max_length is set to 30, but your input_length is only 20. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=10)


...

Your max_length is set to 30, but your input_length is only 15. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=7)


Processing row 24/1196 for model cahya/t5-base-indonesian-summarization-cased


Your max_length is set to 30, but your input_length is only 12. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=6)


Processing row 25/1196 for model cahya/t5-base-indonesian-summarization-cased
Processing row 26/1196 for model cahya/t5-base-indonesian-summarization-cased


: 

In [11]:
del pipe
torch.cuda.empty_cache()

In [12]:

del pipelines
torch.cuda.empty_cache()

In [24]:
save_dir = f'../Data/STAGE 3 PREPROCESSING/SINGARAJA_TEST_STAGE_3_PREPROCESSING_{timestamp}.csv'
df_extracted.to_csv(save_dir, index=False)

In [22]:
df_extracted

,klasifikasi_perkara,terdakwa,hakim,jumlah_saksi,cleaned_barang_bukti,cleaned_dakwaan,total_pidana_penjara_bulan,cahya/t5-base-indonesian-summarization-cased_summarization_dakwaan
0,Kejahatan Perjudian,KETUT SUBAGIA,I Made Bagiarta,3,1 (satu) buah papan bola bergambar 1 (satu) bu...,DAKWAAN PERTAMA - Bahwa Terdakwa KETUT SUBA...,4,DAKWAAN PERTAMA- Bahwa Terdakwa KETUT SUBAGIA ...
1,Kejahatan Perjudian,KOMANG ADITYA,I Made Bagiarta,3,1 (satu) buah papan bola bergambar 1 (satu) bu...,DAKWAAN PERTAMA - Bahwa Terdakwa KETUT SUBA...,4,DAKWAAN PERTAMA- Bahwa Terdakwa KETUT SUBAGIA ...
2,Narkotika,I NYOMAN SUARTA,I Made Bagiarta,3,38 (tiga puluh delapan) potongan pipet plastik...,DAKWAAN KESATU Bahwa ia Terdakwa...,52,DAKWAAN KESATU Bahwa ia Terdakwa I NYOMAN SUAR...
3,Narkotika,RICO JAYADI,I Made Bagiarta,3,1 (satu) buah dompet warna hijau 20 (dua puluh...,DAKWAAN KESATU ...,72,DAKWAAN KESATU Bahwa ia Terdakwa RICO JAYADI p...
4,Narkotika,SANG PUTU WIDIANA,I Gusti Made Juliartawan,4,1(satu) buah HP merk Realme warna hitam putih,Dakwaan Ke Satu - Bahwa terdakwa SANG PUTU W...,48,Dakwaan Ke Satu- Bahwa terdakwa SANG PUTU WIDI...
...,...,...,...,...,...,...,...,...
1191,Perlindungan dan Pengelolaan Lingkungan Hidup,Jumat Ariyanto,Ida Bagus Bama Dewa. P,3,1 (satu) 1 (satu) batang kayu balang-balang be...,Kesatu Pasal 83 ayat (1) huruf b UU RI No. 18...,12,Kesatu Pasal 83 ayat(1) huruf b UU RI No. 18 t...
1192,Kejahatan Perjudian,I KETUT ANA SAPUTRA Alias BENCUT,Sudar,1,36 (tiga puluh enam) lembar kertas rekapan,Perbuatan terdakwa sebagaimana diatur dan dian...,4,Perbuatan terdakwa sebagaimana diatur dan dian...
1193,Penganiayaan,Sahabudin Alias Udin,Ni Luh Suantini,2,1. 1 (satu) buah tabung gas elpiji ukuran 3 (t...,Pasal 351 ayat (1) Kitab Undang-undang Hukum p...,9,Pasal 351 ayat( 1) Kitab Undang-undang Hukum p...
1194,Kejahatan yang Membahayakan Keamananan Umum Ba...,Putu Kristian Damanta,A.A. Sagung Yuni Wulantrisna,4,-1 (satu)unit sepeda motor merk Yamaha Jupiter...,Pertama Pasal 187 ayat (1) KUHP dan Kedua Pa...,24,Pasal 187 ayat(1) KUHP dan Kedua Pasal 363 aya...


In [ ]:
pipe = pipeline("text2text-generation", model="panggi/t5-small-indonesian-summarization-cased", use_fast=False, device="cuda", batch_size=2)

# Track the start time
start_time = time.time()

# Initialize an empty list to store the summaries
summarized_texts = []

# Iterate through the rows of the DataFrame
for index, row in df_extracted.iterrows():
    text = row['cleaned_dakwaan']

    # Chunk the text into smaller pieces
    chunk_size = 512
    chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

    try:
        # Summarize each chunk
        summaries = [pipe(chunk, max_length=30, min_length=20, do_sample=False)[0]['generated_text'] for chunk in chunks]
    except KeyError:
        summaries = [pipe(chunk, max_length=30, min_length=20, do_sample=False)[0]['summary_text'] for chunk in chunks]

    # Combine the summarized chunks into one string
    full_summary = " ".join(summaries)

    # Append the summarized text to the list
    summarized_texts.append(full_summary)

# Add the summarized texts as a new column
df_extracted['second_model_summarization_dakwaan'] = summarized_texts

# Print execution time
print(f"Execution time: {time.time() - start_time} seconds")


### DOCS

In [524]:
from transformers import pipeline

# pipe = pipeline("text2text-generation", model="facebook/mbart-large-50", use_fast=False, device="cuda", batch_size=2)
# pipe = pipeline("text2text-generation", model="indobenchmark/indobart-large")
# pipe = pipeline("summarization", model="mrm8488/t5-base-finetuned-indonesian-summarization")
# pipe = pipeline("text2text-generation", model="google/mt5-large", use_fast=False)

# tensorflow_summarization_models = [
#     "thonyyy/pegasus_indonesian_base-finetune",
#     "thonyyy/pegasus_indonesian_base-pretrain"
# ]

pytorch_summarization_models = [
    # "cahya/bert2bert-indonesian-summarization",
    "cahya/t5-base-indonesian-summarization-cased",
    "panggi/t5-small-indonesian-summarization-cased",
    "interstellarx95/mt5-small-finetuned-indonesian-text-summarization-one-epoch",
    "interstellarx95/mt5-small-finetuned-indonesian-text-summarization-v3",
    # "cahya/bert2gpt-indonesian-summarization",
    "panggi/t5-base-indonesian-summarization-cased",
    "rayendito/mt5-small-finetuned-xl-sum-indonesia",
    "interstellarx95/mt5-small-finetuned-indonesian-text-summarization-three-epochs",
    # "rowjak/bert-indonesian-news-summarization"
]


# tensorflow_summarization_pipes = [
#     pipeline("summarization", model=model, from_tf=True, device=0, batch_size=4)
#     for model in tensorflow_summarization_models
# ]

In [ ]:
def evaluate_meteor(generated, reference):
    # Tokenize the generated and reference text
    tokenized_generated = word_tokenize(generated)
    tokenized_reference = word_tokenize(reference)
    
    # Evaluate METEOR score
    score = meteor_score.meteor_score([tokenized_reference], tokenized_generated)
    return score

def evaluate_rouge(generated, reference):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL', 'rougeLsum'], use_stemmer=True)
    scores = scorer.score(reference, generated)
    return scores

def evaluate_bleu(generated, reference):
    reference_tokens = reference.split()
    generated_tokens = generated.split()
    score = sentence_bleu([reference_tokens], generated_tokens)
    return score

# BERTScore evaluation
def evaluate_bertscore(generated, reference):
    P, R, F1 = score([generated], [reference], lang="en")
    return F1.item()

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Kalea\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Kalea\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Kalea\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
pipe = pipeline("summarization", model="cahya/bert2bert-indonesian-summarization", use_fast=False, device="cuda", batch_size=8)
pipe = pipeline("summarization", model="cahya/t5-base-indonesian-summarization-cased", use_fast=False, device="cuda", batch_size=8)
pipe = pipeline("summarization", model="panggi/t5-small-indonesian-summarization-cased", use_fast=False, device="cuda", batch_size=8)
pipe = pipeline("summarization", model="interstellarx95/mt5-small-finetuned-indonesian-text-summarization-one-epoch", use_fast=False, device="cuda", batch_size=8)
pipe = pipeline("summarization", model="interstellarx95/mt5-small-finetuned-indonesian-text-summarization-v3", use_fast=False, device="cuda", batch_size=8)
pipe = pipeline("summarization", model="cahya/bert2gpt-indonesian-summarization", use_fast=False, device="cuda", batch_size=8)
pipe = pipeline("summarization", model="panggi/t5-base-indonesian-summarization-cased", use_fast=False, device="cuda", batch_size=8)
pipe = pipeline("summarization", model="rayendito/mt5-small-finetuned-xl-sum-indonesia", use_fast=False, device="cuda", batch_size=8)
pipe = pipeline("summarization", model="interstellarx95/mt5-small-finetuned-indonesian-text-summarization-three-epochs", use_fast=False, device="cuda", batch_size=8)
pipe = pipeline("summarization", model="rowjak/bert-indonesian-news-summarization", use_fast=False, device="cuda", batch_size=8)


In [ ]:
import time

text = df_extracted.loc[0,'cleaned_dakwaan']
print(len(text))
reference_summary = """Terdakwa Ketut Subagia dan Komang Aditya pada 2 Juni 2024 sekitar pukul 00.30 WITA di pesisir pantai Gondol, Desa Penyabangan, Kecamatan Gerokgak, Kabupaten Buleleng, Bali, diduga mengadakan permainan judi bola adil secara ilegal. Ketut Subagia bertindak sebagai penyelenggara dan pemodal, sementara Komang Aditya membantu sebagai pekerja yang merapikan uang taruhan serta menyerahkannya kepada pemenang. Permainan melibatkan papan bergambar, bola karet, perlak bergambar, dan cek kayu sebagai pengganti uang taruhan. Barang bukti yang disita meliputi papan bola adil, tas, perlak, bola karet, kayu penyeimbang, bedak bayi, cek kayu dalam berbagai denominasi, lap, dan uang tunai Rp1.108.000. Permainan tersebut dilakukan dengan sistem taruhan yang memberikan hadiah kelipatan sembilan kali lipat dari jumlah taruhan jika menang, namun peserta yang kalah kehilangan uangnya. Aktivitas ini berlangsung selama beberapa jam dengan melibatkan sekitar 20 pemain dan keuntungan digunakan untuk kebutuhan sehari-hari. Namun, kegiatan ini dilakukan tanpa izin resmi dan melanggar hukum, sehingga kedua terdakwa ditangkap dan diancam pidana berdasarkan Pasal 303 KUHP juncto Undang-Undang Nomor 7 Tahun 1974 tentang Penertiban Perjudian."""

for i, model in enumerate(pytorch_summarization_models):
    print(model)
    pipe = pipeline("summarization", model=model, device="cuda", batch_size=4, use_fast=False) 
    chunk_size = 512 

    start_time = time.time()

    chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
    try:
        summaries = [pipe(chunk, max_length=30, min_length=20, do_sample=False)[0]['generated_text'] for chunk in chunks]
    except KeyError:
        summaries = [pipe(chunk, max_length=30, min_length=20, do_sample=False)[0]['summary_text'] for chunk in chunks]

    full_summary = " ".join(summaries)
    end_time = time.time()

    time_taken = end_time - start_time

    print("------------------------------------------------------")

    rouge_scores = evaluate_rouge(full_summary, reference_summary)
    print("ROUGE Scores:", rouge_scores)
    meteor_score_value = evaluate_meteor(full_summary, reference_summary)
    print("METEOR Score:", meteor_score_value)
    bleu_score_value = evaluate_bleu(full_summary, reference_summary)
    print("BLEU Score:", bleu_score_value)
    bertscore_value = evaluate_bertscore(full_summary, reference_summary)
    print("BERTScore F1:", bertscore_value)

    # Display ROUGE scores
    print(len(full_summary))
    print(f"Summary: using {model}: {full_summary}")
    print("------------------------------------------------------")

    del pipe
    torch.cuda.empty_cache()

    print(f"Time taken for summarization: {time_taken:.2f} seconds\n\n")


21579
cahya/t5-base-indonesian-summarization-cased


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


------------------------------------------------------
ROUGE Scores: {'rouge1': Score(precision=0.12121212121212122, recall=0.5847953216374269, fmeasure=0.20080321285140565), 'rouge2': Score(precision=0.04611650485436893, recall=0.2235294117647059, fmeasure=0.07645875251509053), 'rougeL': Score(precision=0.08242424242424243, recall=0.39766081871345027, fmeasure=0.13654618473895586), 'rougeLsum': Score(precision=0.08242424242424243, recall=0.39766081871345027, fmeasure=0.13654618473895586)}
METEOR Score: 0.30761823728726106
BLEU Score: 0.017997592188981122


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore F1: 0.8316876292228699
5486
Summary: using cahya/t5-base-indonesian-summarization-cased: DAKWAAN PERTAMA - Bahwa Terdakwa KETUT SUBAGIA bersama dengan Terdakwa KOMANG ADITYA pada SUBAGIA bersama dengan Terdakwa KOMANG ADITYA pada saat terdakwa KETUT SUBAGIA bersama dengan Terdakwa UT SUBAGIA dan Terdakwa KOMANG ADITYA sedang berlangsung permainan perjudian bola adil di pesisir pantai Gondol, Desa Penya Dua buah bola bergambar, satu (satu) buah tas kain warna abu-abu, satu (satu) buah tas karung warna Sebanyak 44 (empat puluh empat) buah papan cek kayu berwarna biru dengan nomor 20 (duapuluh), 17 (tujuh belas) KETUT SUBAGIA dalam permainan judi bola adil tersebut berperan sebagai penyelenggara yang mengadakan permainan judi bola adil dan juga sebagai penyelenggara yang mengadakan permainan KOMANG ADITYA sebagai pekerja dan merapikan uang serta memberikan uang kepada pemenang permainan bola adil. Bahwa dalam permainan judi bola adil Pasangan uang taruhan judi bola adil 1 (satu)

...

BERTScore F1: 0.8033068180084229
176
Summary: using panggi/t5-base-indonesian-summarization-cased:                 Bahwa di papan bola adil terdapat 4 warna yaitu merah, kuning hijau dan hitam adapun gambarnya yaitu berbentuk segitiga, palang, bola,                          
------------------------------------------------------
Time taken for summarization: 17.71 seconds


rayendito/mt5-small-finetuned-xl-sum-indonesia
------------------------------------------------------
ROUGE Scores: {'rouge1': Score(precision=0.13314037626628075, recall=0.5380116959064327, fmeasure=0.2134570765661253), 'rouge2': Score(precision=0.043478260869565216, recall=0.17647058823529413, fmeasure=0.06976744186046512), 'rougeL': Score(precision=0.06801736613603473, recall=0.27485380116959063, fmeasure=0.10904872389791181), 'rougeLsum': Score(precision=0.06801736613603473, recall=0.27485380116959063, fmeasure=0.10904872389791181)}
METEOR Score: 0.3027963518759939
BLEU Score: 0.01799532323977946


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore F1: 0.8162375688552856
4444
Summary: using rayendito/mt5-small-finetuned-xl-sum-indonesia: Seorang terdakwa kecelakaan pertama - telah mendakwa seorang terdakwa kecelakaan pertama - setelah Polisi dan saksi Polisi dan saksi Polisi dan saksi Polisi dan saksi Polisi dan saksi Polisi dan Seorang pemain bola adil di Bali telah berlangsung permainan perjudian bola adil di pesisir pantai Gondol, Kec Sebuah tas kain warna hitam, 1 (satu) buah tas kain warna hitam, 1 (satu) buah tas Sebuah lembar uang pecahan Rp. 1.108.000,- (satu juta seratus delapan ribu rupiah) terdiri dari Seorang terdakwa keTUT SUBAGIA dalam permainan judi bola adil di Indonesia, telah mendakwa terdakwa keTUT Suba Pemenang permainan bola adil di Indonesia, telah menentukan kemenangan dalam permainan bola adil, karena seorang pemodal Permainan bola adil 1 (satu) buah tas karung warna biru garis merah digunakan untuk pasangan uang Sebuah papan cek kayu berwarna hitam digunakan untuk penyimpan bola adil setelah dig

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore F1: 0.8330612778663635
4361
Summary: using interstellarx95/mt5-small-finetuned-indonesian-text-summarization-three-epochs: Bahwa Terdakwa KETUT SUBAGIA bersama dengan Terdakwa KOMANG ADITYA pada hari Minggu tanggal 02 Juni 2024 udi bola adil dan menjadikan sebagai pencaharian, atau dengan sengaja turut serta dalam suatu perusahaan UT SUBAGIA dan Terdakwa KOMANG ADITYA sedang berlangsung permainan perjudian bola adil di pesisir pantai buah tas kain warna abu-abu, 1 (satu) buah perlak bergambar, 1 (satu) buah perlak uang pecahan Rp. 1.108.000,- (satu juta seratus delapan ribu rupiah) terdiri dari 8 (delapan Terdakwa KETUT SUBAGIA dalam permainan judi bola adil tersebut berperan sebagai penyelenggara yang mengadakan permainan judi bola Pemodal, mengambil uang - uang taruhan yang berada di atas perlak, menggelindingkan bola, sedangkan PAN bola adil 1 (satu) buah perlak bergambar digunakan untuk pasangan uang taruhan bola adil 1 (satu s kain warna hitam digunakan untuk penyimpan b